In [2]:
x1 = {'a':5, 'b':10, 'c':15}
x2 = {'b':20, 'c':30, 'd':50}
x1.update(x2)
x1

{'a': 5, 'b': 20, 'c': 30, 'd': 50}

In [6]:
x1.update(x2)

In [7]:
x1

{'a': 5, 'b': 20, 'c': 30, 'd': 50}

In [5]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import Union, List, Tuple, Any
# from tabulate import tabulate

class AuditLog:
    
    def __init__(self, columns: Union[List[str], Tuple[str]], WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        self.WS_ID = WS_ID
        self.TABLE_NAME_to_check = TABLE_NAME_to_check
        self.AUDIT_TABLE_NAME = AUDIT_TABLE_NAME
        self.LH_ID_to_check = LH_ID_to_check
        self.LH_ID_audit = LH_ID_audit if LH_ID_audit else LH_ID_to_check
        self.schema = schema
        self.fixColumns = {'STARTTIME','ENDTIME','AUDITKEY','STATUS_ACTIVITY'}
        self.columns = tuple(set(columns).union(self.fixColumns))
        
        if self.schema:    
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.schema}/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.schema}/{self.TABLE_NAME_to_check}'
        else:
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.TABLE_NAME_to_check}'
    
        self.log = {column: None for column in self.columns}
        self.log['STARTTIME'] = datetime.now() + timedelta(hours=7)
        self.log['STATUS_ACTIVITY'] = 'Not start'
        
    def initialDetail(self, initConfig: dict[str, Any]):
        assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
        for column in initConfig:
            self.log[column] = initConfig[column]
        
    def __str__(self):
        return str(self.log)
    
class AuditLog_SPC(AuditLog):
    
    def __init__(self, WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        super().__init__(['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE'] ,WS_ID, TABLE_NAME_to_check, AUDIT_TABLE_NAME, LH_ID_to_check, LH_ID_audit, schema)

    def initialDetail(self,  pipelineName: str, pipelineId, TriggerType, TableName, functionName, ):
        super().initialDetail({'PIPELINENAME': pipelineName, 'PIPELINERUNID': pipelineId, 'TRIGGERTYPE': TriggerType, 'TABLE_NAME': TableName, 'FUNCTION_NAME': functionName})

In [ ]:
ad = AuditLog_SPC('SPC_UAT', 'factTest', 'auditTable', 'SilverLH', 'AuditLH')
ad.initialDetail('testPL', '123', 'manual', 'factTest', 'testFunction')

{'STATUS_ACTIVITY': 'Not start', 'TRIGGERTYPE': None, 'ERRORCODE': None, 'STARTTIME': datetime.datetime(2025, 1, 18, 4, 50, 58, 114075), 'ENDTIME': None, 'PIPELINERUNID': None, 'TABLE_NAME': None, 'ERRORMESSAGE': None, 'PIPELINENAME': None, 'AUDITKEY': None, 'COUNTROWSBEFORE': None, 'COUNTROWSAFTER': None, 'FUNCTION_NAME': None}


In [14]:
class FrozenKeysDict:
    def __init__(self, **kwargs):
        self._data = kwargs

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        if key not in self._data:
            raise KeyError(f"Cannot add new key: {key}")
        self._data[key] = value

    def __delitem__(self, key):
        raise KeyError(f"Cannot delete key: {key}")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)

In [18]:
log = FrozenKeysDict(**{column: None for column in ['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE']})

In [21]:
log['PIPELINENAME'] = 'xxx'

In [22]:
log

{'PIPELINENAME': 'xxx', 'PIPELINERUNID': None, 'TRIGGERTYPE': None, 'TABLE_NAME': None, 'FUNCTION_NAME': None, 'COUNTROWSBEFORE': None, 'COUNTROWSAFTER': None, 'ERRORCODE': None, 'ERRORMESSAGE': None}

In [23]:
log['new']='yyy'

KeyError: 'Cannot add new key: new'

In [48]:
class logger:
    def __init__(self, **kwargs):
        self._data = kwargs

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        if key not in self._data:
            raise KeyError(f"Cannot add new key: {key}")
        self._data[key] = value

    def __delitem__(self, key):
        raise KeyError(f"Cannot delete key: {key}")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)

class AuditLog:
    
    def __init__(self, columns: Union[List[str], Tuple[str, ...]], WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        self.WS_ID = WS_ID
        self.TABLE_NAME_to_check = TABLE_NAME_to_check
        self.AUDIT_TABLE_NAME = AUDIT_TABLE_NAME
        self.LH_ID_to_check = LH_ID_to_check
        self.LH_ID_audit = LH_ID_audit if LH_ID_audit else LH_ID_to_check
        self.schema = schema
        self.fixColumns = {'STARTTIME','ENDTIME','AUDITKEY','STATUS_ACTIVITY'}
        self.columns = tuple(set(columns).union(self.fixColumns))
        
        if self.schema:    
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.schema}/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.schema}/{self.TABLE_NAME_to_check}'
        else:
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.TABLE_NAME_to_check}'
    
        self.log = logger(**{column: None for column in self.columns})
        self.log['STARTTIME'] = datetime.now() + timedelta(hours=7)
        self.log['STATUS_ACTIVITY'] = 'Not start'

    def setKey(self, initConfig: dict[str, Any]):
        assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
        for column in initConfig:
            self.log[column] = initConfig[column]
        
    def initialDetail(self, initConfig: dict[str, Any]):
        self.setKey(initConfig)

    def getKey(self):
        return self.columns
    
    def getLog(self):
        return self.log
        
    def __str__(self):
        out = ''
        for key in self.columns:
            out += f'{key}: {self.log[key]}\n'
        return out
    
    def __repr__(self):
        return str(self.log)


class AuditLog_SPC(AuditLog):
    
    def __init__(self, WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        super().__init__(['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE'] ,WS_ID, TABLE_NAME_to_check, AUDIT_TABLE_NAME, LH_ID_to_check, LH_ID_audit, schema)

    def initialDetail(self,  pipelineName: str, pipelineId, TriggerType, TableName, functionName, ):
        super().initialDetail({
            'PIPELINENAME': pipelineName, 
            'PIPELINERUNID': pipelineId, 
            'TRIGGERTYPE': TriggerType, 
            'TABLE_NAME': TableName, 
            'FUNCTION_NAME': functionName
        })

{'STATUS_ACTIVITY': 'Not start', 'TRIGGERTYPE': 'manual', 'ERRORCODE': None, 'STARTTIME': datetime.datetime(2025, 1, 18, 20, 19, 37, 815088), 'ENDTIME': None, 'PIPELINERUNID': '123', 'TABLE_NAME': 'factTest', 'ERRORMESSAGE': None, 'PIPELINENAME': 'testPL', 'AUDITKEY': None, 'COUNTROWSBEFORE': None, 'COUNTROWSAFTER': None, 'FUNCTION_NAME': 'testFunction'}

In [51]:
print(ad)

STATUS_ACTIVITY: Not start
TRIGGERTYPE: manual
ERRORCODE: None
STARTTIME: 2025-01-18 20:19:37.815088
ENDTIME: None
PIPELINERUNID: 123
TABLE_NAME: factTest
ERRORMESSAGE: None
PIPELINENAME: testPL
AUDITKEY: None
COUNTROWSBEFORE: None
COUNTROWSAFTER: None
FUNCTION_NAME: testFunction



In [79]:
import numpy as np
from pyspark.sql.functions import col, when, concat, lit, format_string,sum, upper, substring, expr, current_date, current_timestamp,to_timestamp,concat_ws, isnull, date_format, asc, trim, trunc, date_sub, year,coalesce, count, countDistinct, min, max
from pyspark.sql.types import IntegerType, DecimalType, StringType, LongType, TimestampType, StructType, StructField, DoubleType, FloatType
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import Union, List, Tuple, Any
# from tabulate import tabulate

spark = SparkSession.builder\
        .appName("utils")\
        .getOrCreate()

class logger:
    def __init__(self, **kwargs):
        self._data = kwargs

    def __getitem__(self, key):
        return self._data[key]

    def __setitem__(self, key, value):
        if key not in self._data:
            raise KeyError(f"Cannot add new key: {key}")
        self._data[key] = value

    def __delitem__(self, key):
        raise KeyError(f"Cannot delete key: {key}")

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return repr(self._data)
    
# class Audit:
#     def __init__(self):
#         raise NotImplementedError('This is an abstract class')
    
#     def setKeys(self, initConfig: dict[str, Any]):
#         assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
#         for column in initConfig:
#             self.log[column] = initConfig[column]

        

class AuditLog_Fusion:
    
    def __init__(self, columns: Union[List[str], Tuple[str, ...]], WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        self.WS_ID = WS_ID
        self.TABLE_NAME_to_check = TABLE_NAME_to_check
        self.AUDIT_TABLE_NAME = AUDIT_TABLE_NAME
        self.LH_ID_to_check = LH_ID_to_check
        self.LH_ID_audit = LH_ID_audit if LH_ID_audit else LH_ID_to_check
        self.schema = schema
        self.fixColumns = {'STARTTIME','ENDTIME','AUDITKEY','STATUS_ACTIVITY'}
        self.columns = tuple(set(columns).union(self.fixColumns))
        
        if self.schema:    
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.schema}/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.schema}/{self.TABLE_NAME_to_check}'
        else:
            self.PATH_TO_AUDIT_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_audit}/Tables/{self.AUDIT_TABLE_NAME}'
            self.PATH_TO_CHECKED_TABLE = f'abfss://{self.WS_ID}@onelake.dfs.fabric.microsoft.com/{self.LH_ID_to_check}/Tables/{self.TABLE_NAME_to_check}'
    
        self.log = logger(**{column: None for column in self.columns})
        self.log['STARTTIME'] = datetime.now()
        self.log['STATUS_ACTIVITY'] = 'Not start'

    def setKeys(self, initConfig: dict[str, Any]):
        assert set(initConfig.keys()).issubset(set(self.columns).difference()), f'initConfig must have the columns in {self.columns}'
        for column in initConfig:
            self.log[column] = initConfig[column]

    def setKey(self, key: str, value: Any):
        assert key in self.columns, f'key must be in {self.columns}'
        self.log[key] = value
        
    def initialDetail(self, initConfig: dict[str, Any]):
        self.setKeys(initConfig)

    def getKey(self):
        return self.columns
    
    def getLog(self):
        return self.log
        
    def __str__(self):
        out = ''
        for key in self.columns:
            out += f'{key}: {self.log[key]}\n'
        return out
    
    def __repr__(self):
        return str(self.log)
    
    def endSuccess(self):
        self.log['STATUS_ACTIVITY'] = 'Fail'
        self._endAuditLog()
        print(self)
        
    def endFail(self, errorCode: str, errorMessage: str):
        self.log['STATUS_ACTIVITY'] = 'Fail'
        self.log['ERRORCODE'] = errorCode
        self.log['ERRORMESSAGE'] = errorMessage
        self._endAuditLog()
        print(self)

    def _endAuditLog(self):
        # write to audit table
        self.log['ENDTIME'] = datetime.now()
        df = spark.createDataFrame([self.log])
        df.write.mode('append').save(self.PATH_TO_AUDIT_TABLE)

    def getAuditLogTable(self):
        return spark.read.load(self.PATH_TO_AUDIT_TABLE)
    
    def countBefore(self, df):
        self.log['COUNTROWSBEFORE'] = df.count()

    def countAfter(self, df):
        self.log['COUNTROWSAFTER'] = df.count()

    def getAllPath(self):
        return {'PATH_TO_AUDIT_TABLE':self.PATH_TO_AUDIT_TABLE, 'PATH_TO_CHECKED_TABLE':self.PATH_TO_CHECKED_TABLE}


class AuditLog_SPC(AuditLog_Fusion):
    
    def __init__(self, WS_ID: str, TABLE_NAME_to_check:str, AUDIT_TABLE_NAME:str, LH_ID_to_check: str, LH_ID_audit: str = None, schema: str = None):
        '''
        - if `LH_ID_audit` is not given, it is  LH_ID_to_check automatically, i.e. audit table is in the same lakehouse as that of
        - if using lakehouse with Schema, please provide `schema` parameter
        '''
        super().__init__(['PIPELINENAME', 'PIPELINERUNID', 'TRIGGERTYPE', 'TABLE_NAME', 'FUNCTION_NAME','COUNTROWSBEFORE', 'COUNTROWSAFTER', 'ERRORCODE', 'ERRORMESSAGE'] ,WS_ID, TABLE_NAME_to_check, AUDIT_TABLE_NAME, LH_ID_to_check, LH_ID_audit, schema)

    def initialDetail(self,  pipelineName: str, pipelineId: str, TriggerType: str, TableName: str, functionName: str):
        super().initialDetail({
            'PIPELINENAME': pipelineName, 
            'PIPELINERUNID': pipelineId, 
            'TRIGGERTYPE': TriggerType, 
            'TABLE_NAME': TableName, 
            'FUNCTION_NAME': functionName
        })


In [80]:
ad = AuditLog_SPC(WS_ID = 'SPC_UAT', TABLE_NAME_to_check = 'factTest', AUDIT_TABLE_NAME='auditTable', LH_ID_to_check='SilverLH', LH_ID_audit='AuditLH')
ad.initialDetail(pipelineName = 'testPL', pipelineId = '123', TriggerType = 'manual', TableName = 'factTest', functionName = 'testFunction')
print(ad)

raise FileExistsError('Create you audit table first')

STATUS_ACTIVITY: Not start
TRIGGERTYPE: manual
ERRORCODE: None
STARTTIME: 2025-01-18 13:40:43.660532
ENDTIME: None
PIPELINERUNID: 123
TABLE_NAME: factTest
ERRORMESSAGE: None
PIPELINENAME: testPL
AUDITKEY: None
COUNTROWSBEFORE: None
COUNTROWSAFTER: None
FUNCTION_NAME: testFunction



In [82]:
demoDf = spark.createDataFrame([{'a':1, 'b':2}, {'a':3, 'b':4}])
demoDf.count()

DataFrame[a: bigint, b: bigint]

In [77]:
notebookutils.fs.exists(ad.PATH_TO_AUDIT_TABLE)

STATUS_ACTIVITY: Not start
TRIGGERTYPE: manual
ERRORCODE: None
STARTTIME: 2025-01-18 13:39:16.665795
ENDTIME: None
PIPELINERUNID: 123
TABLE_NAME: factTest
ERRORMESSAGE: None
PIPELINENAME: testPL
AUDITKEY: None
COUNTROWSBEFORE: None
COUNTROWSAFTER: None
FUNCTION_NAME: testFunction



In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# ข้อมูลการตั้งค่าของผู้ส่ง
gmail_user = 'phaphontee.yam@student.mahidol.edu'         # ใส่ที่อยู่อีเมลของคุณ
app_password = 'rziq gbow xoer qlnv'          # ใส่ App Password ที่สร้างไว้

# ข้อมูลของผู้รับและเนื้อหาอีเมล
to = 'phaphontee@fusionsol.com'          # ใส่อีเมลผู้รับ
subject = 'ทดสอบการส่งอีเมลด้วย Python'   # ใส่หัวเรื่อง
body = 'สวัสดีครับ นี่เป็นอีเมลทดสอบที่ส่งด้วย Python!'  # ใส่เนื้อหาของอีเมล

# การสร้างอีเมล
msg = MIMEMultipart()
msg['From'] = gmail_user
msg['To'] = to
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

try:
    # การเชื่อมต่อกับ SMTP ของ Gmail
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()  # เปิดการเข้ารหัส TLS
    server.login(gmail_user, app_password)  # เข้าสู่ระบบด้วยอีเมลและรหัสผ่านแอป
    text = msg.as_string()

    # ส่งอีเมล
    server.sendmail(gmail_user, to, text)
    print("อีเมลถูกส่งสำเร็จ!")

except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")

finally:
    # ปิดการเชื่อมต่อ
    server.quit()



อีเมลถูกส่งสำเร็จ!


In [11]:
import numpy as np
from pyspark.sql.functions import col, when, concat, lit, format_string,sum, upper, substring, expr, current_date, current_timestamp,to_timestamp,concat_ws, isnull, date_format, asc, trim, trunc, date_sub, year,coalesce, count, countDistinct, min, max
from pyspark.sql.types import IntegerType, DecimalType, StringType, LongType, TimestampType, StructType, StructField, DoubleType, FloatType
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from typing import Union, List, Tuple, Any
# from tabulate import tabulate

spark = SparkSession.builder\
        .appName("test")\
        .getOrCreate()

In [12]:
from src.etl.factCPU import CashPickUp_SAP

In [14]:
import pandas as pd

In [ ]:
filename = "1106070_25_03_13.XLSx"
df = pd.read_excel(filename,skiprows=4).fillna("").astype("str").sum(axis=1).str.split("\t",expand=True)
df_col = df.iloc[0].str.replace('.','').str.replace('G/L','G_L')
df = df.iloc[2:]
df.columns = df_col.str.strip().apply(lambda x: '-' if len(x) == 0 else x)
df = df.iloc[:-2]
df = df.drop(columns='-')
df = df[df['Text'].str.contains('CPICK')]
df['AssignmentDate'] = df['Assignment'].str[-8:]
df['DocDate'] = pd.to_datetime(df['Doc Date'].str.replace(".",""),format="%m%d%Y").dt.strftime('%Y%m%d')
df['POSNumber'] = df['Assignment'].str[:5]
df['Amount in doc curr'] = df['Amount in doc curr'].str.strip()
df['DocumentNo'] = df['DocumentNo'].str.strip()
df['AmountCURR'] = df['Amount in doc curr'].str.extract(r'([^\.]+)\..*',expand=False).str.replace(r'[\.,]',"",regex=True)
df['FileName'] = filename
df['ETL_Date'] = datetime.now()
df['YYYYMM_DocDate'] = pd.to_datetime(df['Doc Date'],format='%d.%m.%Y').dt.strftime('%Y%m')

df

,St,Reference,BP,G_L,Cost,Assignment,DocumentNo,Ty,Doc Date,PK,...,Clrng doc,Materia,Text,AssignmentDate,DocDate,POSNumber,AmountCURR,FileName,ETL_Date,YYYYMM_DocDate


In [152]:
filename = "1000020_25_03_13.XLSx"
df = pd.read_excel(filename,skiprows=4).fillna("").astype("str").sum(axis=1).str.split("\t",expand=True)
df_col = df.iloc[0].str.replace('.','').str.replace('G/L','G_L')
df = df.iloc[2:]
df.columns = df_col.str.strip().apply(lambda x: '-' if len(x) == 0 else x)
df = df.iloc[:-2]
df = df.drop(columns='-')
df = df[df['G_L'].str.len()>=1]
df['AssignmentDate'] = df['Assignment'].str[-8:]
df['DocDate'] = pd.to_datetime(df['Doc Date'].str.replace(".",""),format="%m%d%Y").dt.strftime('%Y%m%d')
df['POSNumber'] = df['Assignment'].str[:5]
df['Amount in doc curr'] = df['Amount in doc curr'].str.strip()
df['DocumentNo'] = df['DocumentNo'].str.strip()
df['AmountCURR'] = df['Amount in doc curr'].str.extract(r'([^\.]+)\..*',expand=False).str.replace(r'[\.,]',"",regex=True)
df['FileName'] = filename
df['ETL_Date'] = datetime.now()
df['YYYYMM_DocDate'] = pd.to_datetime(df['Doc Date'],format='%d.%m.%Y').dt.strftime('%Y%m')

df

,St,Reference,BP,G_L,Cost,Assignment,DocumentNo,Ty,Doc Date,PK,...,Clrng doc,Materia,Text,AssignmentDate,DocDate,POSNumber,AmountCURR,FileName,ETL_Date,YYYYMM_DocDate
2,,,0184,1000020,,1001920250301,5106004555,GC,01.03.2025,50,...,,,01.03.2025ปป.ยอดเงินดรอป10019,20250301,20250103,10019,-991,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3,,,0035,1000020,,1003220250301,5106004559,GC,01.03.2025,50,...,,,01.03.2025ปป.ยอดเงินดรอป10032,20250301,20250103,10032,-50,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
4,,,0014,1000020,,1004020250301,5106004561,GC,01.03.2025,40,...,,,01.03.2025ปป.ยอดเงินดรอป10040,20250301,20250103,10040,224167,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
5,,,0059,1000020,,1008520250301,5106004565,GC,01.03.2025,40,...,,,01.03.2025ปป.ยอดเงินดรอป10085,20250301,20250103,10085,136861,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
6,,,0257,1000020,,1010420250301,5106004569,GC,01.03.2025,50,...,,,01.03.2025ปป.ยอดเงินดรอป10104,20250301,20250103,10104,-2584,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,,22220,0031,1000020,,2222020250312,5302021396,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22220,65643,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3897,,22413,0168,1000020,,2241320250312,5302021398,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22413,34419,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3898,,22430,0207,1000020,,2243020250312,5302021399,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22430,71098,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503
3899,,22442,0231,1000020,,2244220250312,5302021400,G3,12.03.2025,40,...,,,CDRP1 เงินสด Drop ลงเซฟ,20250312,20251203,22442,160493,1000020_25_03_13.XLSx,2025-03-31 20:02:23.208218,202503


In [150]:
df[['Doc Date', 'DocDate']]

,Doc Date,DocDate
2,01.03.2025,20250103
3,01.03.2025,20250103
4,01.03.2025,20250103
5,01.03.2025,20250103
6,01.03.2025,20250103
...,...,...
3896,12.03.2025,20251203
3897,12.03.2025,20251203
3898,12.03.2025,20251203
3899,12.03.2025,20251203


In [5]:
pd.DataFrame('''20250324#991310484PMTFLOWCO.CSV
20250331#991310502PMTFLOWCO.CSV
20250328#991410079PMTFLOWCO.CSV
20250331#991404298PMTFLOWCO.CSV
20250331#991310369PMTFLOWCO.CSV
20250327#991310482PMTFLOWCO.CSV
20250324#991310402PMTFLOWCO.CSV
20250327#991310379PMTFLOWCO.CSV
20250327#991310215PMTFLOWCO.CSV
20250331#991310402PMTFLOWCO.CSV
20250329#991410142PMTFLOWCO.CSV
20250326#991310363PMTFLOWCO.CSV
20250327#991310261PMTFLOWCO.CSV
20250330#991310276PMTFLOWCO.CSV
20250322#991310118PMTFLOWCO.CSV
20250330#991310371PMTFLOWCO.CSV
20250401#991310222PMTFLOWCO.CSV
20250323#991404292PMTFLOWCO.CSV
20250325#991310525PMTFLOWCO.CSV
20250319#991310237PMTFLOWCO.CSV
20250327#991310278PMTFLOWCO.CSV
20250326#991404359PMTFLOWCO.CSV
20250321#991310353PMTFLOWCO.CSV
20250325#991310416PMTFLOWCO.CSV
20250331#991310515PMTFLOWCO.CSV
20250325#991310515PMTFLOWCO.CSV
20250324#991310361PMTFLOWCO.CSV
20250320#991404308PMTFLOWCO.CSV
20250320#991310218PMTFLOWCO.CSV
20250331#991310093PMTFLOWCO.CSV
20250327#991310492PMTFLOWCO.CSV
20250324#991310433PMTFLOWCO.CSV
20250328#991410301PMTFLOWCO.CSV
20250324#991310118PMTFLOWCO.CSV
20250327#991404353PMTFLOWCO.CSV
20250323#991404046PMTFLOWCO.CSV
20250402#991310401PMTFLOWCO.CSV
20250330#991310223PMTFLOWCO.CSV
20250322#991310359PMTFLOWCO.CSV
20250323#991310433PMTFLOWCO.CSV
20250325#991310375PMTFLOWCO.CSV
20250322#991310391PMTFLOWCO.CSV
20250320#991310463PMTFLOWCO.CSV
20250402#991310535PMTFLOWCO.CSV
20250401#991310396PMTFLOWCO.CSV
20250321#991310391PMTFLOWCO.CSV
20250321#991310469PMTFLOWCO.CSV
20250327#991410223PMTFLOWCO.CSV
20250323#991310365PMTFLOWCO.CSV
20250321#991310443PMTFLOWCO.CSV
20250402#991310470PMTFLOWCO.CSV
20250331#991310124PMTFLOWCO.CSV
20250325#991310217PMTFLOWCO.CSV
20250326#991310309PMTFLOWCO.CSV
20250402#991310088PMTFLOWCO.CSV
20250402#991310177PMTFLOWCO.CSV
20250328#991310280PMTFLOWCO.CSV
20250322#991410209PMTFLOWCO.CSV
20250329#991310282PMTFLOWCO.CSV
20250327#991310523PMTFLOWCO.CSV
20250319#991310528PMTFLOWCO.CSV
20250321#991310445PMTFLOWCO.CSV
20250330#991310408PMTFLOWCO.CSV
20250326#991310285PMTFLOWCO.CSV
20250320#991310437PMTFLOWCO.CSV
20250323#991310475PMTFLOWCO.CSV
20250328#991310465PMTFLOWCO.CSV
20250401#991310361PMTFLOWCO.CSV
20250326#991310440PMTFLOWCO.CSV
20250321#991310444PMTFLOWCO.CSV
20250326#991310539PMTFLOWCO.CSV
20250326#991310466PMTFLOWCO.CSV
20250325#991310318PMTFLOWCO.CSV
20250324#991310215PMTFLOWCO.CSV
20250402#991310463PMTFLOWCO.CSV
20250402#991310215PMTFLOWCO.CSV
20250323#991310445PMTFLOWCO.CSV
20250322#991310218PMTFLOWCO.CSV
20250321#991310153PMTFLOWCO.CSV
20250320#991310403PMTFLOWCO.CSV
20250325#991404359PMTFLOWCO.CSV
20250326#991410231PMTFLOWCO.CSV
20250328#991310523PMTFLOWCO.CSV
20250402#991310492PMTFLOWCO.CSV
20250324#991310222PMTFLOWCO.CSV
20250331#991310365PMTFLOWCO.CSV
20250329#991310515PMTFLOWCO.CSV
20250322#991310412PMTFLOWCO.CSV
20250328#991310512PMTFLOWCO.CSV
20250321#991310515PMTFLOWCO.CSV
20250328#991310418PMTFLOWCO.CSV
20250322#991310351PMTFLOWCO.CSV
20250402#991310118PMTFLOWCO.CSV
20250331#991310403PMTFLOWCO.CSV
20250321#991310332PMTFLOWCO.CSV
20250402#991404292PMTFLOWCO.CSV
20250402#991310482PMTFLOWCO.CSV
20250325#991310118PMTFLOWCO.CSV
20250319#991310466PMTFLOWCO.CSV
20250319#991310502PMTFLOWCO.CSV
20250329#991410186PMTFLOWCO.CSV
20250331#991310486PMTFLOWCO.CSV
20250319#991310255PMTFLOWCO.CSV
20250401#991310289PMTFLOWCO.CSV
20250401#991310444PMTFLOWCO.CSV
20250329#991310533PMTFLOWCO.CSV
20250330#991310353PMTFLOWCO.CSV
20250326#991310088PMTFLOWCO.CSV
20250327#991310356PMTFLOWCO.CSV
20250326#991310370PMTFLOWCO.CSV
20250324#991310338PMTFLOWCO.CSV
20250402#991310373PMTFLOWCO.CSV
20250331#991410231PMTFLOWCO.CSV
20250401#991310455PMTFLOWCO.CSV
20250320#991310412PMTFLOWCO.CSV
20250402#991410074PMTFLOWCO.CSV
20250325#991310403PMTFLOWCO.CSV
20250322#991410183PMTFLOWCO.CSV
20250324#991410183PMTFLOWCO.CSV
20250327#991410231PMTFLOWCO.CSV
20250402#991310218PMTFLOWCO.CSV
20250321#991410254PMTFLOWCO.CSV
20250329#991310546PMTFLOWCO.CSV
20250321#991410231PMTFLOWCO.CSV
20250319#991310422PMTFLOWCO.CSV
20250328#991310394PMTFLOWCO.CSV
20250324#991410254PMTFLOWCO.CSV
20250323#991310398PMTFLOWCO.CSV
20250322#991310443PMTFLOWCO.CSV
20250326#991310378PMTFLOWCO.CSV
20250326#991310332PMTFLOWCO.CSV
20250324#991310470PMTFLOWCO.CSV
20250331#991303175PMTFLOWCO.CSV
20250326#991310496PMTFLOWCO.CSV
20250329#991410119PMTFLOWCO.CSV
20250321#991310276PMTFLOWCO.CSV
20250330#991310463PMTFLOWCO.CSV
20250331#991310282PMTFLOWCO.CSV
20250330#991310309PMTFLOWCO.CSV
20250331#991310528PMTFLOWCO.CSV
20250321#991310433PMTFLOWCO.CSV
20250319#991310402PMTFLOWCO.CSV
20250329#991310332PMTFLOWCO.CSV
20250331#991310470PMTFLOWCO.CSV
20250331#991310400PMTFLOWCO.CSV
20250319#991310552PMTFLOWCO.CSV
20250330#991310530PMTFLOWCO.CSV
20250402#991310391PMTFLOWCO.CSV
20250322#991310238PMTFLOWCO.CSV
20250401#991310291PMTFLOWCO.CSV
20250319#991310353PMTFLOWCO.CSV
20250330#991310489PMTFLOWCO.CSV
20250329#991310434PMTFLOWCO.CSV
20250321#991310463PMTFLOWCO.CSV
20250328#991310349PMTFLOWCO.CSV
20250327#991310479PMTFLOWCO.CSV
20250322#991310360PMTFLOWCO.CSV
20250322#991404308PMTFLOWCO.CSV
20250324#991310438PMTFLOWCO.CSV
20250330#991310370PMTFLOWCO.CSV
20250326#991310406PMTFLOWCO.CSV
20250326#991310512PMTFLOWCO.CSV
20250322#991310482PMTFLOWCO.CSV
20250327#991310505PMTFLOWCO.CSV
20250402#991310512PMTFLOWCO.CSV
20250323#991310443PMTFLOWCO.CSV
20250326#991310469PMTFLOWCO.CSV
20250325#991310486PMTFLOWCO.CSV
20250326#991310545PMTFLOWCO.CSV
20250320#991310360PMTFLOWCO.CSV
20250321#991310497PMTFLOWCO.CSV
20250325#991310484PMTFLOWCO.CSV
20250330#991310442PMTFLOWCO.CSV
20250323#991310359PMTFLOWCO.CSV
20250324#991310364PMTFLOWCO.CSV
20250320#991310402PMTFLOWCO.CSV
20250326#991404292PMTFLOWCO.CSV
20250320#991310475PMTFLOWCO.CSV
20250329#991310399PMTFLOWCO.CSV
20250328#991310371PMTFLOWCO.CSV
20250401#991310470PMTFLOWCO.CSV
20250327#991310432PMTFLOWCO.CSV
20250322#991310357PMTFLOWCO.CSV
20250402#991310528PMTFLOWCO.CSV
20250320#991310539PMTFLOWCO.CSV
20250322#991404292PMTFLOWCO.CSV
20250324#991310372PMTFLOWCO.CSV
20250402#991404308PMTFLOWCO.CSV
20250323#991310309PMTFLOWCO.CSV
20250324#991310391PMTFLOWCO.CSV
20250330#991310180PMTFLOWCO.CSV
20250330#991310375PMTFLOWCO.CSV
20250326#991310255PMTFLOWCO.CSV
20250330#991310469PMTFLOWCO.CSV
20250321#991410074PMTFLOWCO.CSV
20250322#991310434PMTFLOWCO.CSV
20250329#991310528PMTFLOWCO.CSV
20250327#991310402PMTFLOWCO.CSV
20250323#991410231PMTFLOWCO.CSV
20250401#991310510PMTFLOWCO.CSV
20250320#991310365PMTFLOWCO.CSV
20250323#991410142PMTFLOWCO.CSV
20250321#991310462PMTFLOWCO.CSV
20250326#991310480PMTFLOWCO.CSV
20250323#991310093PMTFLOWCO.CSV
20250327#991310400PMTFLOWCO.CSV
20250324#991310533PMTFLOWCO.CSV
20250321#991404359PMTFLOWCO.CSV
20250324#991310546PMTFLOWCO.CSV
20250331#991310489PMTFLOWCO.CSV
20250321#991404624PMTFLOWCO.CSV
20250330#991310228PMTFLOWCO.CSV
20250324#991310453PMTFLOWCO.CSV
20250327#991410074PMTFLOWCO.CSV
20250324#991310538PMTFLOWCO.CSV
20250331#991310359PMTFLOWCO.CSV
20250325#991304009PMTFLOWCO.CSV
20250328#991310480PMTFLOWCO.CSV
20250331#991310537PMTFLOWCO.CSV
20250320#991310348PMTFLOWCO.CSV
20250328#991310356PMTFLOWCO.CSV
20250321#991310088PMTFLOWCO.CSV
20250331#991310418PMTFLOWCO.CSV
20250328#991310437PMTFLOWCO.CSV
20250327#991310464PMTFLOWCO.CSV
20250324#991310515PMTFLOWCO.CSV
20250324#991310408PMTFLOWCO.CSV
20250325#991310223PMTFLOWCO.CSV
20250321#991310375PMTFLOWCO.CSV
20250401#991404046PMTFLOWCO.CSV
20250324#991310218PMTFLOWCO.CSV
20250322#991310165PMTFLOWCO.CSV
20250321#991310364PMTFLOWCO.CSV
20250401#991310348PMTFLOWCO.CSV
20250327#991310537PMTFLOWCO.CSV
20250322#991310498PMTFLOWCO.CSV
20250326#991310502PMTFLOWCO.CSV
20250324#991310376PMTFLOWCO.CSV
20250402#991404232PMTFLOWCO.CSV
20250331#991310405PMTFLOWCO.CSV
20250324#991310552PMTFLOWCO.CSV
20250402#991310442PMTFLOWCO.CSV
20250330#991310487PMTFLOWCO.CSV
20250325#991310222PMTFLOWCO.CSV
20250322#991310332PMTFLOWCO.CSV
20250327#991310442PMTFLOWCO.CSV
20250327#991310501PMTFLOWCO.CSV
20250322#991310309PMTFLOWCO.CSV
20250326#991310498PMTFLOWCO.CSV
20250322#991310438PMTFLOWCO.CSV
20250321#991310409PMTFLOWCO.CSV
20250326#991310523PMTFLOWCO.CSV
20250325#991310440PMTFLOWCO.CSV
20250319#991310506PMTFLOWCO.CSV
20250320#991404353PMTFLOWCO.CSV
20250328#991310207PMTFLOWCO.CSV
20250325#991310371PMTFLOWCO.CSV
20250320#991310222PMTFLOWCO.CSV
20250319#991310357PMTFLOWCO.CSV
20250319#991310530PMTFLOWCO.CSV
20250329#991310511PMTFLOWCO.CSV
20250320#991310496PMTFLOWCO.CSV
20250328#991310546PMTFLOWCO.CSV
20250402#991410183PMTFLOWCO.CSV
20250329#991310472PMTFLOWCO.CSV
20250402#991303286PMTFLOWCO.CSV
20250327#991310396PMTFLOWCO.CSV
20250327#991404046PMTFLOWCO.CSV
20250320#991404287PMTFLOWCO.CSV
20250331#991310462PMTFLOWCO.CSV
20250321#991310453PMTFLOWCO.CSV
20250329#991310409PMTFLOWCO.CSV
20250331#991310258PMTFLOWCO.CSV
20250322#991310255PMTFLOWCO.CSV
20250402#991410063PMTFLOWCO.CSV
20250330#991404287PMTFLOWCO.CSV
20250401#991310492PMTFLOWCO.CSV
20250329#991304009PMTFLOWCO.CSV
20250320#991310261PMTFLOWCO.CSV
20250325#991310401PMTFLOWCO.CSV
20250326#991310528PMTFLOWCO.CSV
20250322#991404353PMTFLOWCO.CSV
20250322#991310356PMTFLOWCO.CSV
20250331#991310223PMTFLOWCO.CSV
20250320#991310472PMTFLOWCO.CSV
20250319#991310072PMTFLOWCO.CSV
20250326#991310097PMTFLOWCO.CSV
20250331#991410117PMTFLOWCO.CSV
20250401#991303930PMTFLOWCO.CSV
20250326#991404646PMTFLOWCO.CSV
20250324#991410144PMTFLOWCO.CSV
20250331#991404359PMTFLOWCO.CSV
20250329#991310405PMTFLOWCO.CSV
20250331#991310351PMTFLOWCO.CSV
20250330#991310430PMTFLOWCO.CSV
20250320#991310327PMTFLOWCO.CSV
20250326#991310404PMTFLOWCO.CSV
20250330#991410079PMTFLOWCO.CSV
20250326#991310465PMTFLOWCO.CSV
20250321#991304009PMTFLOWCO.CSV
20250324#991310432PMTFLOWCO.CSV
20250331#991310309PMTFLOWCO.CSV
20250324#991410117PMTFLOWCO.CSV
20250323#991404184PMTFLOWCO.CSV
20250322#991310518PMTFLOWCO.CSV
20250401#991310365PMTFLOWCO.CSV
20250330#991404646PMTFLOWCO.CSV
20250323#991310378PMTFLOWCO.CSV
20250326#991404308PMTFLOWCO.CSV
20250320#991310528PMTFLOWCO.CSV
20250402#991410186PMTFLOWCO.CSV
20250320#991310072PMTFLOWCO.CSV
20250328#991410142PMTFLOWCO.CSV
20250331#991410079PMTFLOWCO.CSV
20250318#991404646PMTFLOWCO.CSV
20250331#991310237PMTFLOWCO.CSV
20250319#991410151PMTFLOWCO.CSV
20250319#991310409PMTFLOWCO.CSV
20250401#991310402PMTFLOWCO.CSV
20250330#991310391PMTFLOWCO.CSV
20250323#991410063PMTFLOWCO.CSV
20250402#991410151PMTFLOWCO.CSV
20250323#991310356PMTFLOWCO.CSV
20250330#991310537PMTFLOWCO.CSV
20250323#991310546PMTFLOWCO.CSV
20250326#991310463PMTFLOWCO.CSV
20250331#991410151PMTFLOWCO.CSV
20250320#991310282PMTFLOWCO.CSV
20250402#991310453PMTFLOWCO.CSV
20250402#991310408PMTFLOWCO.CSV
20250324#991310153PMTFLOWCO.CSV
20250324#991410222PMTFLOWCO.CSV
20250322#991410144PMTFLOWCO.CSV
20250320#991310461PMTFLOWCO.CSV
20250328#991310237PMTFLOWCO.CSV
20250327#991310440PMTFLOWCO.CSV
20250330#991310409PMTFLOWCO.CSV
20250325#991310408PMTFLOWCO.CSV
20250329#991310518PMTFLOWCO.CSV
20250327#991310125PMTFLOWCO.CSV
20250330#991310515PMTFLOWCO.CSV
20250323#991310379PMTFLOWCO.CSV
20250328#991310416PMTFLOWCO.CSV
20250328#991310326PMTFLOWCO.CSV
20250326#991310546PMTFLOWCO.CSV
20250324#991310349PMTFLOWCO.CSV
20250331#991310378PMTFLOWCO.CSV
20250325#991310278PMTFLOWCO.CSV
20250322#991310545PMTFLOWCO.CSV
20250329#991310538PMTFLOWCO.CSV
20250330#991310484PMTFLOWCO.CSV
20250320#991310255PMTFLOWCO.CSV
20250326#991310222PMTFLOWCO.CSV
20250319#991410222PMTFLOWCO.CSV
20250322#991310348PMTFLOWCO.CSV
20250321#991310361PMTFLOWCO.CSV
20250401#991310180PMTFLOWCO.CSV
20250326#991310464PMTFLOWCO.CSV
20250323#991410079PMTFLOWCO.CSV
20250325#991310373PMTFLOWCO.CSV
20250320#991310445PMTFLOWCO.CSV
20250324#991310510PMTFLOWCO.CSV
20250324#991303175PMTFLOWCO.CSV
20250330#991310378PMTFLOWCO.CSV
20250401#991310496PMTFLOWCO.CSV
20250321#991310360PMTFLOWCO.CSV
20250324#991310475PMTFLOWCO.CSV
20250331#991310530PMTFLOWCO.CSV
20250402#991410223PMTFLOWCO.CSV
20250319#991310392PMTFLOWCO.CSV
20250402#991310239PMTFLOWCO.CSV
20250320#991410301PMTFLOWCO.CSV
20250329#991310430PMTFLOWCO.CSV
20250402#991310348PMTFLOWCO.CSV
20250328#991410151PMTFLOWCO.CSV
20250325#991404308PMTFLOWCO.CSV
20250402#991310261PMTFLOWCO.CSV
20250321#991404287PMTFLOWCO.CSV
20250331#991310218PMTFLOWCO.CSV
20250324#991404353PMTFLOWCO.CSV
20250326#991410151PMTFLOWCO.CSV
20250319#991404178PMTFLOWCO.CSV
20250325#991404184PMTFLOWCO.CSV
20250320#991410183PMTFLOWCO.CSV
20250325#991310228PMTFLOWCO.CSV
20250320#991310239PMTFLOWCO.CSV
20250322#991404287PMTFLOWCO.CSV
20250330#991310433PMTFLOWCO.CSV
20250327#991310496PMTFLOWCO.CSV
20250322#991310405PMTFLOWCO.CSV
20250329#991410189PMTFLOWCO.CSV
20250402#991310153PMTFLOWCO.CSV
20250324#991310511PMTFLOWCO.CSV
20250402#991310480PMTFLOWCO.CSV
20250331#991310207PMTFLOWCO.CSV
20250327#991310349PMTFLOWCO.CSV
20250329#991310537PMTFLOWCO.CSV
20250331#991310460PMTFLOWCO.CSV
20250328#991310528PMTFLOWCO.CSV
20250330#991410151PMTFLOWCO.CSV
20250320#991310405PMTFLOWCO.CSV
20250331#991310278PMTFLOWCO.CSV
20250324#991310507PMTFLOWCO.CSV
20250323#991310406PMTFLOWCO.CSV
20250401#991310318PMTFLOWCO.CSV
20250319#991310418PMTFLOWCO.CSV
20250322#991310207PMTFLOWCO.CSV
20250327#991310338PMTFLOWCO.CSV
20250322#991310180PMTFLOWCO.CSV
20250402#991410254PMTFLOWCO.CSV
20250401#991310338PMTFLOWCO.CSV
20250325#991310202PMTFLOWCO.CSV
20250328#991310533PMTFLOWCO.CSV
20250327#991310405PMTFLOWCO.CSV
20250320#991310466PMTFLOWCO.CSV
20250402#991310479PMTFLOWCO.CSV
20250401#991310373PMTFLOWCO.CSV
20250319#991310177PMTFLOWCO.CSV
20250329#991310523PMTFLOWCO.CSV
20250319#991310261PMTFLOWCO.CSV
20250401#991310518PMTFLOWCO.CSV
20250323#991310545PMTFLOWCO.CSV
20250319#991310394PMTFLOWCO.CSV
20250331#991310348PMTFLOWCO.CSV
20250326#991404624PMTFLOWCO.CSV
20250328#991310124PMTFLOWCO.CSV
20250330#991410189PMTFLOWCO.CSV
20250328#991404184PMTFLOWCO.CSV
20250327#991310406PMTFLOWCO.CSV
20250401#991310285PMTFLOWCO.CSV
20250402#991310382PMTFLOWCO.CSV
20250326#991310511PMTFLOWCO.CSV
20250327#991404308PMTFLOWCO.CSV
20250323#991310452PMTFLOWCO.CSV
20250401#991310430PMTFLOWCO.CSV
20250325#991310452PMTFLOWCO.CSV
20250322#991310511PMTFLOWCO.CSV
20250319#991404353PMTFLOWCO.CSV
20250320#991410223PMTFLOWCO.CSV
20250328#991310472PMTFLOWCO.CSV
20250329#991404287PMTFLOWCO.CSV
20250325#991310332PMTFLOWCO.CSV
20250401#991310401PMTFLOWCO.CSV
20250330#991310088PMTFLOWCO.CSV
20250402#991310363PMTFLOWCO.CSV
20250330#991310464PMTFLOWCO.CSV
20250321#991303175PMTFLOWCO.CSV
20250330#991310177PMTFLOWCO.CSV
20250327#991310165PMTFLOWCO.CSV
20250322#991410231PMTFLOWCO.CSV
20250322#991310469PMTFLOWCO.CSV
20250328#991310258PMTFLOWCO.CSV
20250330#991310379PMTFLOWCO.CSV
20250322#991310223PMTFLOWCO.CSV
20250320#991310278PMTFLOWCO.CSV
20250327#991310357PMTFLOWCO.CSV
20250401#991310223PMTFLOWCO.CSV
20250322#991404178PMTFLOWCO.CSV
20250319#991310482PMTFLOWCO.CSV
20250324#991310280PMTFLOWCO.CSV
20250402#991404298PMTFLOWCO.CSV
20250324#991310505PMTFLOWCO.CSV
20250324#991310351PMTFLOWCO.CSV
20250401#991310376PMTFLOWCO.CSV
20250327#991410079PMTFLOWCO.CSV
20250329#991310359PMTFLOWCO.CSV
20250331#991310177PMTFLOWCO.CSV
20250330#991310399PMTFLOWCO.CSV
20250321#991310432PMTFLOWCO.CSV
20250330#991310403PMTFLOWCO.CSV
20250331#991410105PMTFLOWCO.CSV
20250402#991310459PMTFLOWCO.CSV
20250323#991310413PMTFLOWCO.CSV
20250330#991310518PMTFLOWCO.CSV
20250321#991310434PMTFLOWCO.CSV
20250326#991310322PMTFLOWCO.CSV
20250320#991310379PMTFLOWCO.CSV
20250401#991410117PMTFLOWCO.CSV
20250401#991310546PMTFLOWCO.CSV
20250331#991310420PMTFLOWCO.CSV
20250323#991310326PMTFLOWCO.CSV
20250402#991310523PMTFLOWCO.CSV
20250328#991310088PMTFLOWCO.CSV
20250329#991310400PMTFLOWCO.CSV
20250323#991310405PMTFLOWCO.CSV
20250323#991310444PMTFLOWCO.CSV
20250326#991404387PMTFLOWCO.CSV
20250320#991310492PMTFLOWCO.CSV
20250328#991310118PMTFLOWCO.CSV
20250402#991310498PMTFLOWCO.CSV
20250327#991310363PMTFLOWCO.CSV
20250330#991310376PMTFLOWCO.CSV
20250328#991404287PMTFLOWCO.CSV
20250325#991310415PMTFLOWCO.CSV
20250402#991310515PMTFLOWCO.CSV
20250327#991310475PMTFLOWCO.CSV
20250327#991310545PMTFLOWCO.CSV
20250324#991310382PMTFLOWCO.CSV
20250327#991310382PMTFLOWCO.CSV
20250325#991310538PMTFLOWCO.CSV
20250402#991310518PMTFLOWCO.CSV
20250329#991310348PMTFLOWCO.CSV
20250319#991310382PMTFLOWCO.CSV
20250328#991410105PMTFLOWCO.CSV
20250319#991310378PMTFLOWCO.CSV
20250324#991310360PMTFLOWCO.CSV
20250322#991310535PMTFLOWCO.CSV
20250325#991310261PMTFLOWCO.CSV
20250321#991404184PMTFLOWCO.CSV
20250402#991310501PMTFLOWCO.CSV
20250323#991310153PMTFLOWCO.CSV
20250402#991310416PMTFLOWCO.CSV
20250324#991310404PMTFLOWCO.CSV
20250320#991310413PMTFLOWCO.CSV
20250322#991310239PMTFLOWCO.CSV
20250319#991310398PMTFLOWCO.CSV
20250326#991310261PMTFLOWCO.CSV
20250326#991310409PMTFLOWCO.CSV
20250323#991310487PMTFLOWCO.CSV
20250323#991310223PMTFLOWCO.CSV
20250401#991310356PMTFLOWCO.CSV
20250330#991310356PMTFLOWCO.CSV
20250320#991310326PMTFLOWCO.CSV
20250329#991310510PMTFLOWCO.CSV
20250320#991310215PMTFLOWCO.CSV
20250329#991310475PMTFLOWCO.CSV
20250321#991310255PMTFLOWCO.CSV
20250328#991310218PMTFLOWCO.CSV
20250402#991310552PMTFLOWCO.CSV
20250327#991404292PMTFLOWCO.CSV
20250328#991310379PMTFLOWCO.CSV
20250325#991310378PMTFLOWCO.CSV
20250328#991310412PMTFLOWCO.CSV
20250331#991310153PMTFLOWCO.CSV
20250401#991410074PMTFLOWCO.CSV
20250319#991310512PMTFLOWCO.CSV
20250322#991310533PMTFLOWCO.CSV
20250319#991310452PMTFLOWCO.CSV
20250329#991310401PMTFLOWCO.CSV
20250330#991310506PMTFLOWCO.CSV
20250401#991310382PMTFLOWCO.CSV
20250329#991310327PMTFLOWCO.CSV
20250319#991310465PMTFLOWCO.CSV
20250329#991310351PMTFLOWCO.CSV
20250325#991310418PMTFLOWCO.CSV
20250329#991410105PMTFLOWCO.CSV
20250331#991310404PMTFLOWCO.CSV
20250324#991404232PMTFLOWCO.CSV
20250323#991310409PMTFLOWCO.CSV
20250325#991404191PMTFLOWCO.CSV
20250401#991310165PMTFLOWCO.CSV
20250320#991310401PMTFLOWCO.CSV
20250331#991404046PMTFLOWCO.CSV
20250328#991310439PMTFLOWCO.CSV
20250326#991304009PMTFLOWCO.CSV
20250319#991310538PMTFLOWCO.CSV
20250326#991310472PMTFLOWCO.CSV
20250321#991410119PMTFLOWCO.CSV
20250330#991310207PMTFLOWCO.CSV
20250321#991410144PMTFLOWCO.CSV
20250401#991404871PMTFLOWCO.CSV
20250320#991310289PMTFLOWCO.CSV
20250402#991310460PMTFLOWCO.CSV
20250331#991310512PMTFLOWCO.CSV
20250329#991310432PMTFLOWCO.CSV
20250320#991310332PMTFLOWCO.CSV
20250331#991310501PMTFLOWCO.CSV
20250324#991410223PMTFLOWCO.CSV
20250327#991310535PMTFLOWCO.CSV
20250328#991310357PMTFLOWCO.CSV
20250323#991310228PMTFLOWCO.CSV
20250327#991310439PMTFLOWCO.CSV
20250321#991404292PMTFLOWCO.CSV
20250321#991310479PMTFLOWCO.CSV
20250322#991404359PMTFLOWCO.CSV
20250326#991310402PMTFLOWCO.CSV
20250325#991310258PMTFLOWCO.CSV
20250329#991310338PMTFLOWCO.CSV
20250324#991310420PMTFLOWCO.CSV
20250330#991310475PMTFLOWCO.CSV
20250402#991310465PMTFLOWCO.CSV
20250402#991310326PMTFLOWCO.CSV
20250329#991310378PMTFLOWCO.CSV
20250402#991310180PMTFLOWCO.CSV
20250330#991310338PMTFLOWCO.CSV
20250325#991310400PMTFLOWCO.CSV
20250319#991310445PMTFLOWCO.CSV
20250321#991310501PMTFLOWCO.CSV
20250329#991310502PMTFLOWCO.CSV
20250323#991404178PMTFLOWCO.CSV
20250326#991310177PMTFLOWCO.CSV
20250329#991310470PMTFLOWCO.CSV
20250323#991310472PMTFLOWCO.CSV
20250323#991310318PMTFLOWCO.CSV
20250327#991310409PMTFLOWCO.CSV
20250324#991410119PMTFLOWCO.CSV
20250401#991310512PMTFLOWCO.CSV
20250326#991310382PMTFLOWCO.CSV
20250329#991310437PMTFLOWCO.CSV
20250321#991310399PMTFLOWCO.CSV
20250330#991404308PMTFLOWCO.CSV
20250321#991310228PMTFLOWCO.CSV
20250328#991310382PMTFLOWCO.CSV
20250329#991410231PMTFLOWCO.CSV
20250320#991310228PMTFLOWCO.CSV
20250330#991310202PMTFLOWCO.CSV
20250328#991310348PMTFLOWCO.CSV
20250331#991310072PMTFLOWCO.CSV
20250324#991310459PMTFLOWCO.CSV
20250323#991310489PMTFLOWCO.CSV
20250326#991310282PMTFLOWCO.CSV
20250401#991310276PMTFLOWCO.CSV
20250326#991310351PMTFLOWCO.CSV
20250323#991310515PMTFLOWCO.CSV
20250327#991310460PMTFLOWCO.CSV
20250329#991310093PMTFLOWCO.CSV
20250402#991310486PMTFLOWCO.CSV
20250326#991310445PMTFLOWCO.CSV
20250325#991310523PMTFLOWCO.CSV
20250401#991310475PMTFLOWCO.CSV
20250329#991310480PMTFLOWCO.CSV
20250321#991410147PMTFLOWCO.CSV
20250329#991310207PMTFLOWCO.CSV
20250322#991410301PMTFLOWCO.CSV
20250319#991310415PMTFLOWCO.CSV
20250325#991310402PMTFLOWCO.CSV
20250326#991410147PMTFLOWCO.CSV
20250319#991410209PMTFLOWCO.CSV
20250331#991310280PMTFLOWCO.CSV
20250320#991310180PMTFLOWCO.CSV
20250331#991410142PMTFLOWCO.CSV
20250319#991404624PMTFLOWCO.CSV
20250325#991410189PMTFLOWCO.CSV
20250321#991310318PMTFLOWCO.CSV
20250325#991310369PMTFLOWCO.CSV
20250402#991310455PMTFLOWCO.CSV
20250325#991310498PMTFLOWCO.CSV
20250402#991310349PMTFLOWCO.CSV
20250329#991310222PMTFLOWCO.CSV
20250329#991410079PMTFLOWCO.CSV
20250330#991410183PMTFLOWCO.CSV
20250331#991404646PMTFLOWCO.CSV
20250328#991310215PMTFLOWCO.CSV
20250319#991310280PMTFLOWCO.CSV
20250323#991410254PMTFLOWCO.CSV
20250318#991310291PMTFLOWCO.CSV
20250331#991310442PMTFLOWCO.CSV
20250330#991410105PMTFLOWCO.CSV
20250322#991310496PMTFLOWCO.CSV
20250324#991310097PMTFLOWCO.CSV
20250319#991310284PMTFLOWCO.CSV
20250322#991310257PMTFLOWCO.CSV
20250322#991310289PMTFLOWCO.CSV
20250330#991310507PMTFLOWCO.CSV
20250319#991310462PMTFLOWCO.CSV
20250325#991310482PMTFLOWCO.CSV
20250327#991310153PMTFLOWCO.CSV
20250323#991310482PMTFLOWCO.CSV
20250328#991310361PMTFLOWCO.CSV
20250330#991310217PMTFLOWCO.CSV
20250329#991310088PMTFLOWCO.CSV
20250319#991310433PMTFLOWCO.CSV
20250328#991310318PMTFLOWCO.CSV
20250328#991310433PMTFLOWCO.CSV
20250320#991310530PMTFLOWCO.CSV
20250323#991410301PMTFLOWCO.CSV
20250401#991404184PMTFLOWCO.CSV
20250325#991310284PMTFLOWCO.CSV
20250319#991310501PMTFLOWCO.CSV
20250319#991310165PMTFLOWCO.CSV
20250325#991310280PMTFLOWCO.CSV
20250322#991310546PMTFLOWCO.CSV
20250325#991310479PMTFLOWCO.CSV
20250325#991310502PMTFLOWCO.CSV
20250329#991310370PMTFLOWCO.CSV
20250401#991310238PMTFLOWCO.CSV
20250329#991310438PMTFLOWCO.CSV
20250320#991310165PMTFLOWCO.CSV
20250325#991310349PMTFLOWCO.CSV
20250328#991310440PMTFLOWCO.CSV
20250328#991310461PMTFLOWCO.CSV
20250324#991310371PMTFLOWCO.CSV
20250322#991310376PMTFLOWCO.CSV
20250327#991310515PMTFLOWCO.CSV
20250323#991310484PMTFLOWCO.CSV
20250323#991310285PMTFLOWCO.CSV
20250322#991310370PMTFLOWCO.CSV
20250325#991310238PMTFLOWCO.CSV
20250331#991310364PMTFLOWCO.CSV
20250402#991310372PMTFLOWCO.CSV
20250402#991310507PMTFLOWCO.CSV
20250329#991404191PMTFLOWCO.CSV
20250402#991310223PMTFLOWCO.CSV
20250402#991310356PMTFLOWCO.CSV
20250324#991310353PMTFLOWCO.CSV
20250325#991310215PMTFLOWCO.CSV
20250402#991310165PMTFLOWCO.CSV
20250320#991303175PMTFLOWCO.CSV
20250322#991310530PMTFLOWCO.CSV
20250327#991310282PMTFLOWCO.CSV
20250324#991410301PMTFLOWCO.CSV
20250319#991310472PMTFLOWCO.CSV
20250328#991404292PMTFLOWCO.CSV
20250321#991310363PMTFLOWCO.CSV
20250322#991310363PMTFLOWCO.CSV
20250330#991310215PMTFLOWCO.CSV
20250321#991310365PMTFLOWCO.CSV
20250320#991310501PMTFLOWCO.CSV
20250328#991303930PMTFLOWCO.CSV
20250328#991310498PMTFLOWCO.CSV
20250323#991310394PMTFLOWCO.CSV
20250320#991310370PMTFLOWCO.CSV
20250326#991310357PMTFLOWCO.CSV
20250402#991310434PMTFLOWCO.CSV
20250327#991310528PMTFLOWCO.CSV
20250323#991410186PMTFLOWCO.CSV
20250401#991310418PMTFLOWCO.CSV
20250318#991404871PMTFLOWCO.CSV
20250322#991310502PMTFLOWCO.CSV
20250331#991310338PMTFLOWCO.CSV
20250322#991310217PMTFLOWCO.CSV
20250324#991303930PMTFLOWCO.CSV
20250401#991310440PMTFLOWCO.CSV
20250322#991310415PMTFLOWCO.CSV
20250321#991310202PMTFLOWCO.CSV
20250324#991410063PMTFLOWCO.CSV
20250330#991310480PMTFLOWCO.CSV
20250401#991310541PMTFLOWCO.CSV
20250331#991404292PMTFLOWCO.CSV
20250326#991310507PMTFLOWCO.CSV
20250319#991310486PMTFLOWCO.CSV
20250402#991310280PMTFLOWCO.CSV
20250326#991310482PMTFLOWCO.CSV
20250320#991310489PMTFLOWCO.CSV
20250331#991310437PMTFLOWCO.CSV
20250327#991410209PMTFLOWCO.CSV
20250331#991310332PMTFLOWCO.CSV
20250327#991310373PMTFLOWCO.CSV
20250331#991310415PMTFLOWCO.CSV
20250331#991310469PMTFLOWCO.CSV
20250402#991310530PMTFLOWCO.CSV
20250323#991310418PMTFLOWCO.CSV
20250326#991310443PMTFLOWCO.CSV
20250326#991310289PMTFLOWCO.CSV
20250321#991310535PMTFLOWCO.CSV
20250323#991310528PMTFLOWCO.CSV
20250319#991310444PMTFLOWCO.CSV
20250401#991310399PMTFLOWCO.CSV
20250325#991310218PMTFLOWCO.CSV
20250319#991404359PMTFLOWCO.CSV
20250401#991310097PMTFLOWCO.CSV
20250401#991404353PMTFLOWCO.CSV
20250322#991310475PMTFLOWCO.CSV
20250330#991310440PMTFLOWCO.CSV
20250320#991404298PMTFLOWCO.CSV
20250326#991310348PMTFLOWCO.CSV
20250402#991310409PMTFLOWCO.CSV
20250322#991310422PMTFLOWCO.CSV
20250328#991310459PMTFLOWCO.CSV
20250323#991310541PMTFLOWCO.CSV
20250323#991310462PMTFLOWCO.CSV
20250320#991410105PMTFLOWCO.CSV
20250325#991310357PMTFLOWCO.CSV
20250322#991310285PMTFLOWCO.CSV
20250401#991404292PMTFLOWCO.CSV
20250330#991310406PMTFLOWCO.CSV
20250320#991410254PMTFLOWCO.CSV
20250325#991310327PMTFLOWCO.CSV
20250330#991303286PMTFLOWCO.CSV
20250330#991310497PMTFLOWCO.CSV
20250325#991310464PMTFLOWCO.CSV
20250319#991310434PMTFLOWCO.CSV
20250323#991310370PMTFLOWCO.CSV
20250331#991410063PMTFLOWCO.CSV
20250319#991310484PMTFLOWCO.CSV
20250327#991310539PMTFLOWCO.CSV
20250402#991310258PMTFLOWCO.CSV
20250330#991310364PMTFLOWCO.CSV
20250330#991310392PMTFLOWCO.CSV
20250326#991410144PMTFLOWCO.CSV
20250326#991404184PMTFLOWCO.CSV
20250326#991310228PMTFLOWCO.CSV
20250402#991310472PMTFLOWCO.CSV
20250323#991310218PMTFLOWCO.CSV
20250323#991310537PMTFLOWCO.CSV
20250319#991310356PMTFLOWCO.CSV
20250322#991310276PMTFLOWCO.CSV
20250327#991310371PMTFLOWCO.CSV
20250320#991310415PMTFLOWCO.CSV
20250401#991310506PMTFLOWCO.CSV
20250322#991310539PMTFLOWCO.CSV
20250322#991310072PMTFLOWCO.CSV
20250327#991310378PMTFLOWCO.CSV
20250331#991310412PMTFLOWCO.CSV
20250319#991410223PMTFLOWCO.CSV
20250319#991310420PMTFLOWCO.CSV
20250328#991310424PMTFLOWCO.CSV
20250320#991310533PMTFLOWCO.CSV
20250331#991310261PMTFLOWCO.CSV
20250402#991310438PMTFLOWCO.CSV
20250401#991310326PMTFLOWCO.CSV
20250325#991310465PMTFLOWCO.CSV
20250331#991310434PMTFLOWCO.CSV
20250330#991310285PMTFLOWCO.CSV
20250330#991310238PMTFLOWCO.CSV
20250328#991310398PMTFLOWCO.CSV
20250324#991410105PMTFLOWCO.CSV
20250324#991310398PMTFLOWCO.CSV
20250319#991310289PMTFLOWCO.CSV
20250323#991310422PMTFLOWCO.CSV
20250326#991410189PMTFLOWCO.CSV
20250327#991310375PMTFLOWCO.CSV
20250327#991310486PMTFLOWCO.CSV
20250319#991310464PMTFLOWCO.CSV
20250328#991310460PMTFLOWCO.CSV
20250320#991310093PMTFLOWCO.CSV
20250401#991310153PMTFLOWCO.CSV
20250320#991310394PMTFLOWCO.CSV
20250323#991410222PMTFLOWCO.CSV
20250322#991310430PMTFLOWCO.CSV
20250326#991404178PMTFLOWCO.CSV
20250325#991410063PMTFLOWCO.CSV
20250331#991310361PMTFLOWCO.CSV
20250326#991310379PMTFLOWCO.CSV
20250329#991310535PMTFLOWCO.CSV
20250325#991310472PMTFLOWCO.CSV
20250319#991310228PMTFLOWCO.CSV
20250329#991310479PMTFLOWCO.CSV
20250401#991310228PMTFLOWCO.CSV
20250330#991310237PMTFLOWCO.CSV
20250401#991310484PMTFLOWCO.CSV
20250327#991310404PMTFLOWCO.CSV
20250321#991310489PMTFLOWCO.CSV
20250326#991310453PMTFLOWCO.CSV
20250329#991310255PMTFLOWCO.CSV
20250330#991310165PMTFLOWCO.CSV
20250325#991310439PMTFLOWCO.CSV
20250320#991310375PMTFLOWCO.CSV
20250319#991310475PMTFLOWCO.CSV
20250326#991310479PMTFLOWCO.CSV
20250325#991310469PMTFLOWCO.CSV
20250322#991310153PMTFLOWCO.CSV
20250324#991310466PMTFLOWCO.CSV
20250327#991310413PMTFLOWCO.CSV
20250321#991310093PMTFLOWCO.CSV
20250323#991310284PMTFLOWCO.CSV
20250323#991310261PMTFLOWCO.CSV
20250326#991310165PMTFLOWCO.CSV
20250323#991310097PMTFLOWCO.CSV
20250328#991310255PMTFLOWCO.CSV
20250402#991310093PMTFLOWCO.CSV
20250331#991310538PMTFLOWCO.CSV
20250330#991310439PMTFLOWCO.CSV
20250328#991310291PMTFLOWCO.CSV
20250328#991310430PMTFLOWCO.CSV
20250331#991310510PMTFLOWCO.CSV
20250320#991303286PMTFLOWCO.CSV
20250330#991310396PMTFLOWCO.CSV
20250322#991310365PMTFLOWCO.CSV
20250329#991310497PMTFLOWCO.CSV
20250319#991310406PMTFLOWCO.CSV
20250319#991310371PMTFLOWCO.CSV
20250325#991404387PMTFLOWCO.CSV
20250327#991310512PMTFLOWCO.CSV
20250401#991310258PMTFLOWCO.CSV
20250329#991310257PMTFLOWCO.CSV
20250323#991310327PMTFLOWCO.CSV
20250401#991310480PMTFLOWCO.CSV
20250331#991310439PMTFLOWCO.CSV
20250331#991310497PMTFLOWCO.CSV
20250402#991310291PMTFLOWCO.CSV
20250330#991310496PMTFLOWCO.CSV
20250330#991404178PMTFLOWCO.CSV
20250322#991310177PMTFLOWCO.CSV
20250402#991310365PMTFLOWCO.CSV
20250327#991310376PMTFLOWCO.CSV
20250331#991310284PMTFLOWCO.CSV
20250320#991310309PMTFLOWCO.CSV
20250322#991310462PMTFLOWCO.CSV
20250321#991310415PMTFLOWCO.CSV
20250327#991310392PMTFLOWCO.CSV
20250322#991310349PMTFLOWCO.CSV
20250323#991310535PMTFLOWCO.CSV
20250321#991310218PMTFLOWCO.CSV
20250319#991404308PMTFLOWCO.CSV
20250327#991310533PMTFLOWCO.CSV
20250326#991310280PMTFLOWCO.CSV
20250326#991310401PMTFLOWCO.CSV
20250328#991310276PMTFLOWCO.CSV
20250328#991310364PMTFLOWCO.CSV
20250326#991310475PMTFLOWCO.CSV
20250326#991404353PMTFLOWCO.CSV
20250319#991310396PMTFLOWCO.CSV
20250326#991410254PMTFLOWCO.CSV
20250321#991310413PMTFLOWCO.CSV
20250326#991310535PMTFLOWCO.CSV
20250320#991310118PMTFLOWCO.CSV
20250322#991310396PMTFLOWCO.CSV
20250321#991310528PMTFLOWCO.CSV
20250321#991310418PMTFLOWCO.CSV
20250325#991310255PMTFLOWCO.CSV
20250321#991310422PMTFLOWCO.CSV
20250322#991310453PMTFLOWCO.CSV
20250329#991310125PMTFLOWCO.CSV
20250325#991310489PMTFLOWCO.CSV
20250321#991310327PMTFLOWCO.CSV
20250325#991310239PMTFLOWCO.CSV
20250329#991310413PMTFLOWCO.CSV
20250328#991310530PMTFLOWCO.CSV
20250326#991310257PMTFLOWCO.CSV
20250401#991404232PMTFLOWCO.CSV
20250402#991410301PMTFLOWCO.CSV
20250328#991310284PMTFLOWCO.CSV
20250329#991404308PMTFLOWCO.CSV
20250329#991310486PMTFLOWCO.CSV
20250323#991310465PMTFLOWCO.CSV
20250324#991404387PMTFLOWCO.CSV
20250401#991410119PMTFLOWCO.CSV
20250324#991310284PMTFLOWCO.CSV
20250402#991310222PMTFLOWCO.CSV
20250322#991410079PMTFLOWCO.CSV
20250328#991410231PMTFLOWCO.CSV
20250323#991310453PMTFLOWCO.CSV
20250326#991310202PMTFLOWCO.CSV
20250329#991310218PMTFLOWCO.CSV
20250402#991310440PMTFLOWCO.CSV
20250323#991404353PMTFLOWCO.CSV
20250320#991310438PMTFLOWCO.CSV
20250322#991310420PMTFLOWCO.CSV
20250327#991310217PMTFLOWCO.CSV
20250319#991310365PMTFLOWCO.CSV
20250402#991310097PMTFLOWCO.CSV
20250402#991310072PMTFLOWCO.CSV
20250331#991310541PMTFLOWCO.CSV
20250401#991410120PMTFLOWCO.CSV
20250320#991310453PMTFLOWCO.CSV
20250320#991310462PMTFLOWCO.CSV
20250329#991310463PMTFLOWCO.CSV
20250326#991310515PMTFLOWCO.CSV
20250323#991404359PMTFLOWCO.CSV
20250402#991310125PMTFLOWCO.CSV
20250320#991303930PMTFLOWCO.CSV
20250325#991310420PMTFLOWCO.CSV
20250319#991310318PMTFLOWCO.CSV
20250320#991310440PMTFLOWCO.CSV
20250322#991310459PMTFLOWCO.CSV
20250328#991310363PMTFLOWCO.CSV
20250321#991310552PMTFLOWCO.CSV
20250327#991303930PMTFLOWCO.CSV
20250326#991310438PMTFLOWCO.CSV
20250329#991310496PMTFLOWCO.CSV
20250328#991310463PMTFLOWCO.CSV
20250323#991410151PMTFLOWCO.CSV
20250322#991310408PMTFLOWCO.CSV
20250402#991310539PMTFLOWCO.CSV
20250327#991404298PMTFLOWCO.CSV
20250322#991310338PMTFLOWCO.CSV
20250401#991310379PMTFLOWCO.CSV
20250330#991310404PMTFLOWCO.CSV
20250320#991310434PMTFLOWCO.CSV
20250326#991310396PMTFLOWCO.CSV
20250323#991310289PMTFLOWCO.CSV
20250401#991310322PMTFLOWCO.CSV
20250321#991310394PMTFLOWCO.CSV
20250328#991410223PMTFLOWCO.CSV
20250320#991410231PMTFLOWCO.CSV
20250402#991410117PMTFLOWCO.CSV
20250329#991310539PMTFLOWCO.CSV
20250321#991310487PMTFLOWCO.CSV
20250325#991310309PMTFLOWCO.CSV
20250322#991310528PMTFLOWCO.CSV
20250326#991310416PMTFLOWCO.CSV
20250402#991310394PMTFLOWCO.CSV
20250401#991310375PMTFLOWCO.CSV
20250318#991404184PMTFLOWCO.CSV
20250401#991310404PMTFLOWCO.CSV
20250401#991310282PMTFLOWCO.CSV
20250330#991310486PMTFLOWCO.CSV
20250327#991310502PMTFLOWCO.CSV
20250321#991310541PMTFLOWCO.CSV
20250327#991410120PMTFLOWCO.CSV
20250323#991410105PMTFLOWCO.CSV
20250330#991310538PMTFLOWCO.CSV
20250329#991310462PMTFLOWCO.CSV
20250330#991310432PMTFLOWCO.CSV
20250331#991310357PMTFLOWCO.CSV
20250325#991310180PMTFLOWCO.CSV
20250330#991310412PMTFLOWCO.CSV
20250319#991310349PMTFLOWCO.CSV
20250402#991310237PMTFLOWCO.CSV
20250324#991310403PMTFLOWCO.CSV
20250330#991310369PMTFLOWCO.CSV
20250401#991310525PMTFLOWCO.CSV
20250319#991303286PMTFLOWCO.CSV
20250324#991310413PMTFLOWCO.CSV
20250328#991310470PMTFLOWCO.CSV
20250321#991303286PMTFLOWCO.CSV
20250320#991310452PMTFLOWCO.CSV
20250401#991310408PMTFLOWCO.CSV
20250321#991310539PMTFLOWCO.CSV
20250326#991310533PMTFLOWCO.CSV
20250321#991310207PMTFLOWCO.CSV
20250324#991310365PMTFLOWCO.CSV
20250327#991410189PMTFLOWCO.CSV
20250327#991310398PMTFLOWCO.CSV
20250326#991310452PMTFLOWCO.CSV
20250319#991310469PMTFLOWCO.CSV
20250323#991310372PMTFLOWCO.CSV
20250319#991310207PMTFLOWCO.CSV
20250330#991310452PMTFLOWCO.CSV
20250319#991310291PMTFLOWCO.CSV
20250321#991404298PMTFLOWCO.CSV
20250320#991310284PMTFLOWCO.CSV
20250321#991310351PMTFLOWCO.CSV
20250320#991310338PMTFLOWCO.CSV
20250325#991310413PMTFLOWCO.CSV
20250322#991310202PMTFLOWCO.CSV
20250331#991310496PMTFLOWCO.CSV
20250402#991310437PMTFLOWCO.CSV
20250329#991310404PMTFLOWCO.CSV
20250330#991410147PMTFLOWCO.CSV
20250401#991310217PMTFLOWCO.CSV
20250324#991310180PMTFLOWCO.CSV
20250330#991410186PMTFLOWCO.CSV
20250319#991310326PMTFLOWCO.CSV
20250330#991310239PMTFLOWCO.CSV
20250321#991310282PMTFLOWCO.CSV
20250325#991310462PMTFLOWCO.CSV
20250329#991310284PMTFLOWCO.CSV
20250328#991404298PMTFLOWCO.CSV
20250329#991410209PMTFLOWCO.CSV
20250326#991310412PMTFLOWCO.CSV
20250326#991310218PMTFLOWCO.CSV
20250330#991310373PMTFLOWCO.CSV
20250326#991310375PMTFLOWCO.CSV
20250319#991310180PMTFLOWCO.CSV
20250328#991310453PMTFLOWCO.CSV
20250331#991310370PMTFLOWCO.CSV
20250321#991310223PMTFLOWCO.CSV
20250401#991310438PMTFLOWCO.CSV
20250327#991310525PMTFLOWCO.CSV
20250324#991310434PMTFLOWCO.CSV
20250325#991410209PMTFLOWCO.CSV
20250331#991310422PMTFLOWCO.CSV
20250401#991310392PMTFLOWCO.CSV
20250330#991310258PMTFLOWCO.CSV
20250328#991310413PMTFLOWCO.CSV
20250330#991310255PMTFLOWCO.CSV
20250325#991404287PMTFLOWCO.CSV
20250321#991310124PMTFLOWCO.CSV
20250327#991310177PMTFLOWCO.CSV
20250323#991310376PMTFLOWCO.CSV
20250325#991410120PMTFLOWCO.CSV
20250325#991310463PMTFLOWCO.CSV
20250319#991404387PMTFLOWCO.CSV
20250325#991410074PMTFLOWCO.CSV
20250321#991310498PMTFLOWCO.CSV
20250328#991404308PMTFLOWCO.CSV
20250326#991310459PMTFLOWCO.CSV
20250321#991410183PMTFLOWCO.CSV
20250327#991310291PMTFLOWCO.CSV
20250327#991310276PMTFLOWCO.CSV
20250327#991310218PMTFLOWCO.CSV
20250329#991310452PMTFLOWCO.CSV
20250402#991310546PMTFLOWCO.CSV
20250323#991310348PMTFLOWCO.CSV
20250320#991310357PMTFLOWCO.CSV
20250331#991310498PMTFLOWCO.CSV
20250319#991310461PMTFLOWCO.CSV
20250327#991310408PMTFLOWCO.CSV
20250329#991310165PMTFLOWCO.CSV
20250401#991410079PMTFLOWCO.CSV
20250402#991310360PMTFLOWCO.CSV
20250328#991310202PMTFLOWCO.CSV
20250330#991310546PMTFLOWCO.CSV
20250324#991310093PMTFLOWCO.CSV
20250401#991410183PMTFLOWCO.CSV
20250330#991310418PMTFLOWCO.CSV
20250321#991310530PMTFLOWCO.CSV
20250402#991310432PMTFLOWCO.CSV
20250321#991310452PMTFLOWCO.CSV
20250320#991310469PMTFLOWCO.CSV
20250328#991310510PMTFLOWCO.CSV
20250331#991310371PMTFLOWCO.CSV
20250330#991310153PMTFLOWCO.CSV
20250319#991410206PMTFLOWCO.CSV
20250319#991310537PMTFLOWCO.CSV
20250328#991310177PMTFLOWCO.CSV
20250326#991310541PMTFLOWCO.CSV
20250328#991310479PMTFLOWCO.CSV
20250401#991410144PMTFLOWCO.CSV
20250328#991310511PMTFLOWCO.CSV
20250326#991310484PMTFLOWCO.CSV
20250327#991310470PMTFLOWCO.CSV
20250328#991310228PMTFLOWCO.CSV
20250330#991310284PMTFLOWCO.CSV
20250401#991310515PMTFLOWCO.CSV
20250322#991310093PMTFLOWCO.CSV
20250325#991310433PMTFLOWCO.CSV
20250326#991310207PMTFLOWCO.CSV
20250401#991310489PMTFLOWCO.CSV
20250322#991310373PMTFLOWCO.CSV
20250322#991410105PMTFLOWCO.CSV
20250330#991310413PMTFLOWCO.CSV
20250322#991310480PMTFLOWCO.CSV
20250328#991404624PMTFLOWCO.CSV
20250324#991310487PMTFLOWCO.CSV
20250401#991310363PMTFLOWCO.CSV
20250321#991310546PMTFLOWCO.CSV
20250329#991310505PMTFLOWCO.CSV
20250325#991310359PMTFLOWCO.CSV
20250331#991310276PMTFLOWCO.CSV
20250401#991310415PMTFLOWCO.CSV
20250327#991310365PMTFLOWCO.CSV
20250330#991410254PMTFLOWCO.CSV
20250327#991310546PMTFLOWCO.CSV
20250401#991310093PMTFLOWCO.CSV
20250401#991310309PMTFLOWCO.CSV
20250320#991310408PMTFLOWCO.CSV
20250331#991310322PMTFLOWCO.CSV
20250402#991310332PMTFLOWCO.CSV
20250329#991310097PMTFLOWCO.CSV
20250322#991310382PMTFLOWCO.CSV
20250321#991404353PMTFLOWCO.CSV
20250326#991410120PMTFLOWCO.CSV
20250331#991310545PMTFLOWCO.CSV
20250321#991310177PMTFLOWCO.CSV
20250322#991310375PMTFLOWCO.CSV
20250325#991310518PMTFLOWCO.CSV
20250322#991310541PMTFLOWCO.CSV
20250320#991404359PMTFLOWCO.CSV
20250319#991310505PMTFLOWCO.CSV
20250323#991310498PMTFLOWCO.CSV
20250325#991310391PMTFLOWCO.CSV
20250330#991310445PMTFLOWCO.CSV
20250326#991310356PMTFLOWCO.CSV
20250324#991410147PMTFLOWCO.CSV
20250401#991310359PMTFLOWCO.CSV
20250329#991310360PMTFLOWCO.CSV
20250330#991310528PMTFLOWCO.CSV
20250325#991310535PMTFLOWCO.CSV
20250321#991310461PMTFLOWCO.CSV
20250401#991310255PMTFLOWCO.CSV
20250324#991310223PMTFLOWCO.CSV
20250330#991310332PMTFLOWCO.CSV
20250328#991310360PMTFLOWCO.CSV
20250325#991310097PMTFLOWCO.CSV
20250320#991310433PMTFLOWCO.CSV
20250323#991310276PMTFLOWCO.CSV
20250320#991310537PMTFLOWCO.CSV
20250401#991310434PMTFLOWCO.CSV
20250321#991310538PMTFLOWCO.CSV
20250402#991310415PMTFLOWCO.CSV
20250325#991410147PMTFLOWCO.CSV
20250402#991310505PMTFLOWCO.CSV
20250330#991310402PMTFLOWCO.CSV
20250327#991310284PMTFLOWCO.CSV
20250319#991310498PMTFLOWCO.CSV
20250329#991404178PMTFLOWCO.CSV
20250401#991310372PMTFLOWCO.CSV
20250325#991310406PMTFLOWCO.CSV
20250327#991310364PMTFLOWCO.CSV
20250323#991410189PMTFLOWCO.CSV
20250327#991310466PMTFLOWCO.CSV
20250321#991310348PMTFLOWCO.CSV
20250319#991410120PMTFLOWCO.CSV
20250323#991310391PMTFLOWCO.CSV
20250329#991310177PMTFLOWCO.CSV
20250320#991310153PMTFLOWCO.CSV
20250319#991310376PMTFLOWCO.CSV
20250325#991310437PMTFLOWCO.CSV
20250320#991310318PMTFLOWCO.CSV
20250325#991310404PMTFLOWCO.CSV
20250319#991310125PMTFLOWCO.CSV
20250323#991310332PMTFLOWCO.CSV
20250323#991310088PMTFLOWCO.CSV
20250325#991310276PMTFLOWCO.CSV
20250329#991310261PMTFLOWCO.CSV
20250326#991404232PMTFLOWCO.CSV
20250326#991310442PMTFLOWCO.CSV
20250320#991310443PMTFLOWCO.CSV
20250329#991310364PMTFLOWCO.CSV
20250328#991310376PMTFLOWCO.CSV
20250327#991310223PMTFLOWCO.CSV
20250331#991310353PMTFLOWCO.CSV
20250401#991310538PMTFLOWCO.CSV
20250402#991310371PMTFLOWCO.CSV
20250402#991310282PMTFLOWCO.CSV
20250331#991310399PMTFLOWCO.CSV
20250401#991310406PMTFLOWCO.CSV
20250329#991310439PMTFLOWCO.CSV
20250323#991310469PMTFLOWCO.CSV
20250321#991410186PMTFLOWCO.CSV
20250323#991310322PMTFLOWCO.CSV
20250329#991310202PMTFLOWCO.CSV
20250323#991310518PMTFLOWCO.CSV
20250326#991310405PMTFLOWCO.CSV
20250324#991310238PMTFLOWCO.CSV
20250402#991310537PMTFLOWCO.CSV
20250324#991310452PMTFLOWCO.CSV
20250401#991310369PMTFLOWCO.CSV
20250321#991310470PMTFLOWCO.CSV
20250324#991310502PMTFLOWCO.CSV
20250319#991310440PMTFLOWCO.CSV
20250319#991310438PMTFLOWCO.CSV
20250330#991310093PMTFLOWCO.CSV
20250401#991310462PMTFLOWCO.CSV
20250327#991310239PMTFLOWCO.CSV
20250324#991310217PMTFLOWCO.CSV
20250401#991310442PMTFLOWCO.CSV
20250320#991410074PMTFLOWCO.CSV
20250320#991310400PMTFLOWCO.CSV
20250331#991304009PMTFLOWCO.CSV
20250322#991310466PMTFLOWCO.CSV
20250328#991310484PMTFLOWCO.CSV
20250325#991310475PMTFLOWCO.CSV
20250325#991310541PMTFLOWCO.CSV
20250325#991404646PMTFLOWCO.CSV
20250329#991310392PMTFLOWCO.CSV
20250320#991310359PMTFLOWCO.CSV
20250325#991310382PMTFLOWCO.CSV
20250325#991310356PMTFLOWCO.CSV
20250323#991310459PMTFLOWCO.CSV
20250326#991410222PMTFLOWCO.CSV
20250327#991310118PMTFLOWCO.CSV
20250323#991310222PMTFLOWCO.CSV
20250331#991310525PMTFLOWCO.CSV
20250323#991310180PMTFLOWCO.CSV
20250402#991310462PMTFLOWCO.CSV
20250327#991404287PMTFLOWCO.CSV
20250326#991310237PMTFLOWCO.CSV
20250326#991310327PMTFLOWCO.CSV
20250322#991310222PMTFLOWCO.CSV
20250330#991310444PMTFLOWCO.CSV
20250326#991310371PMTFLOWCO.CSV
20250328#991310278PMTFLOWCO.CSV
20250327#991310461PMTFLOWCO.CSV
20250323#991310399PMTFLOWCO.CSV
20250326#991310432PMTFLOWCO.CSV
20250323#991310400PMTFLOWCO.CSV
20250324#991310359PMTFLOWCO.CSV
20250321#991404191PMTFLOWCO.CSV
20250329#991310318PMTFLOWCO.CSV
20250330#991310125PMTFLOWCO.CSV
20250325#991410231PMTFLOWCO.CSV
20250402#991310217PMTFLOWCO.CSV
20250321#991310237PMTFLOWCO.CSV
20250323#991310291PMTFLOWCO.CSV
20250401#991410223PMTFLOWCO.CSV
20250323#991310375PMTFLOWCO.CSV
20250329#991310459PMTFLOWCO.CSV
20250321#991310326PMTFLOWCO.CSV
20250324#991310506PMTFLOWCO.CSV
20250326#991404287PMTFLOWCO.CSV
20250329#991310285PMTFLOWCO.CSV
20250321#991310464PMTFLOWCO.CSV
20250331#991310392PMTFLOWCO.CSV
20250324#991310394PMTFLOWCO.CSV
20250331#991310360PMTFLOWCO.CSV
20250328#991310444PMTFLOWCO.CSV
20250331#991310552PMTFLOWCO.CSV
20250325#991310460PMTFLOWCO.CSV
20250323#991310338PMTFLOWCO.CSV
20250319#991310332PMTFLOWCO.CSV
20250324#991310125PMTFLOWCO.CSV
20250326#991310153PMTFLOWCO.CSV
20250322#991310124PMTFLOWCO.CSV
20250326#991310326PMTFLOWCO.CSV
20250325#991404178PMTFLOWCO.CSV
20250325#991310434PMTFLOWCO.CSV
20250322#991310515PMTFLOWCO.CSV
20250320#991410209PMTFLOWCO.CSV
20250327#991410144PMTFLOWCO.CSV
20250326#991310505PMTFLOWCO.CSV
20250331#991310239PMTFLOWCO.CSV
20250328#991310402PMTFLOWCO.CSV
20250329#991404298PMTFLOWCO.CSV
20250326#991410117PMTFLOWCO.CSV
20250402#991410105PMTFLOWCO.CSV
20250325#991310539PMTFLOWCO.CSV
20250327#991310370PMTFLOWCO.CSV
20250323#991310552PMTFLOWCO.CSV
20250320#991410144PMTFLOWCO.CSV
20250320#991310404PMTFLOWCO.CSV
20250320#991310418PMTFLOWCO.CSV
20250328#991310378PMTFLOWCO.CSV
20250327#991310359PMTFLOWCO.CSV
20250319#991310442PMTFLOWCO.CSV
20250401#991410147PMTFLOWCO.CSV
20250328#991410144PMTFLOWCO.CSV
20250325#991410105PMTFLOWCO.CSV
20250322#991310361PMTFLOWCO.CSV
20250328#991410209PMTFLOWCO.CSV
20250326#991310434PMTFLOWCO.CSV
20250328#991310443PMTFLOWCO.CSV
20250327#991310258PMTFLOWCO.CSV
20250321#991310370PMTFLOWCO.CSV
20250331#991310511PMTFLOWCO.CSV
20250323#991310360PMTFLOWCO.CSV
20250320#991310376PMTFLOWCO.CSV
20250319#991310359PMTFLOWCO.CSV
20250325#991310511PMTFLOWCO.CSV
20250320#991310378PMTFLOWCO.CSV
20250331#991310461PMTFLOWCO.CSV
20250319#991310400PMTFLOWCO.CSV
20250320#991310124PMTFLOWCO.CSV
20250325#991310546PMTFLOWCO.CSV
20250327#991310093PMTFLOWCO.CSV
20250322#991310280PMTFLOWCO.CSV
20250326#991410209PMTFLOWCO.CSV
20250323#991310118PMTFLOWCO.CSV
20250325#991310497PMTFLOWCO.CSV
20250330#991310326PMTFLOWCO.CSV
20250402#991310278PMTFLOWCO.CSV
20250324#991310437PMTFLOWCO.CSV
20250328#991310257PMTFLOWCO.CSV
20250320#991310237PMTFLOWCO.CSV
20250319#991310507PMTFLOWCO.CSV
20250319#991310479PMTFLOWCO.CSV
20250324#991404046PMTFLOWCO.CSV
20250322#991410147PMTFLOWCO.CSV
20250322#991310489PMTFLOWCO.CSV
20250319#991310470PMTFLOWCO.CSV
20250330#991310401PMTFLOWCO.CSV
20250319#991310401PMTFLOWCO.CSV
20250325#991310512PMTFLOWCO.CSV
20250323#991310282PMTFLOWCO.CSV
20250401#991310332PMTFLOWCO.CSV
20250327#991410147PMTFLOWCO.CSV
20250323#991310165PMTFLOWCO.CSV
20250331#991310492PMTFLOWCO.CSV
20250319#991310238PMTFLOWCO.CSV
20250324#991310443PMTFLOWCO.CSV
20250330#991410144PMTFLOWCO.CSV
20250329#991404353PMTFLOWCO.CSV
20250329#991410183PMTFLOWCO.CSV
20250319#991310408PMTFLOWCO.CSV
20250325#991310445PMTFLOWCO.CSV
20250324#991310496PMTFLOWCO.CSV
20250329#991310228PMTFLOWCO.CSV
20250325#991310455PMTFLOWCO.CSV
20250330#991310289PMTFLOWCO.CSV
20250320#991404046PMTFLOWCO.CSV
20250401#991310479PMTFLOWCO.CSV
20250322#991310278PMTFLOWCO.CSV
20250331#991310440PMTFLOWCO.CSV
20250325#991310361PMTFLOWCO.CSV
20250331#991310291PMTFLOWCO.CSV
20250321#991310533PMTFLOWCO.CSV
20250323#991310479PMTFLOWCO.CSV
20250402#991310444PMTFLOWCO.CSV
20250401#991310487PMTFLOWCO.CSV
20250401#991310501PMTFLOWCO.CSV
20250330#991310416PMTFLOWCO.CSV
20250319#991310515PMTFLOWCO.CSV
20250324#991410120PMTFLOWCO.CSV
20250326#991410074PMTFLOWCO.CSV
20250402#991310322PMTFLOWCO.CSV
20250320#991310207PMTFLOWCO.CSV
20250329#991410147PMTFLOWCO.CSV
20250402#991310289PMTFLOWCO.CSV
20250402#991310353PMTFLOWCO.CSV
20250327#991310518PMTFLOWCO.CSV
20250324#991310124PMTFLOWCO.CSV
20250326#991410186PMTFLOWCO.CSV
20250330#991404184PMTFLOWCO.CSV
20250330#991410142PMTFLOWCO.CSV
20250320#991310392PMTFLOWCO.CSV
20250402#991310403PMTFLOWCO.CSV
20250324#991310207PMTFLOWCO.CSV
20250401#991310420PMTFLOWCO.CSV
20250328#991310487PMTFLOWCO.CSV
20250329#991303930PMTFLOWCO.CSV
20250320#991310552PMTFLOWCO.CSV
20250323#991310215PMTFLOWCO.CSV
20250319#991310443PMTFLOWCO.CSV
20250320#991310546PMTFLOWCO.CSV
20250401#991310482PMTFLOWCO.CSV
20250320#991310545PMTFLOWCO.CSV
20250327#991310511PMTFLOWCO.CSV
20250319#991310363PMTFLOWCO.CSV
20250331#991410223PMTFLOWCO.CSV
20250322#991303286PMTFLOWCO.CSV
20250402#991310402PMTFLOWCO.CSV
20250329#991310415PMTFLOWCO.CSV
20250331#991310165PMTFLOWCO.CSV
20250321#991404178PMTFLOWCO.CSV
20250323#991310207PMTFLOWCO.CSV
20250401#991310530PMTFLOWCO.CSV
20250328#991310420PMTFLOWCO.CSV
20250324#991310523PMTFLOWCO.CSV
20250329#991310482PMTFLOWCO.CSV
20250320#991310349PMTFLOWCO.CSV
20250323#991310512PMTFLOWCO.CSV
20250402#991410231PMTFLOWCO.CSV
20250330#991310460PMTFLOWCO.CSV
20250320#991310177PMTFLOWCO.CSV
20250331#991310097PMTFLOWCO.CSV
20250320#991310484PMTFLOWCO.CSV
20250329#991310506PMTFLOWCO.CSV
20250327#991410222PMTFLOWCO.CSV
20250328#991310375PMTFLOWCO.CSV
20250325#991310501PMTFLOWCO.CSV
20250319#991310539PMTFLOWCO.CSV
20250324#991310088PMTFLOWCO.CSV
20250401#991310498PMTFLOWCO.CSV
20250322#991310442PMTFLOWCO.CSV
20250321#991310442PMTFLOWCO.CSV
20250319#991310217PMTFLOWCO.CSV
20250402#991310452PMTFLOWCO.CSV
20250319#991310370PMTFLOWCO.CSV
20250331#991310202PMTFLOWCO.CSV
20250323#991310463PMTFLOWCO.CSV
20250327#991310434PMTFLOWCO.CSV
20250329#991310072PMTFLOWCO.CSV
20250331#991410189PMTFLOWCO.CSV
20250328#991310482PMTFLOWCO.CSV
20250330#991410301PMTFLOWCO.CSV
20250325#991310282PMTFLOWCO.CSV
20250319#991310453PMTFLOWCO.CSV
20250329#991410074PMTFLOWCO.CSV
20250319#991310489PMTFLOWCO.CSV
20250323#991410209PMTFLOWCO.CSV
20250321#991410079PMTFLOWCO.CSV
20250330#991310365PMTFLOWCO.CSV
20250402#991310412PMTFLOWCO.CSV
20250402#991310207PMTFLOWCO.CSV
20250322#991404184PMTFLOWCO.CSV
20250326#991310538PMTFLOWCO.CSV
20250326#991310238PMTFLOWCO.CSV
20250324#991310318PMTFLOWCO.CSV
20250329#991310461PMTFLOWCO.CSV
20250319#991410186PMTFLOWCO.CSV
20250324#991410189PMTFLOWCO.CSV
20250325#991310399PMTFLOWCO.CSV
20250328#991310506PMTFLOWCO.CSV
20250402#991310424PMTFLOWCO.CSV
20250319#991310545PMTFLOWCO.CSV
20250321#991310465PMTFLOWCO.CSV
20250320#991410120PMTFLOWCO.CSV
20250331#991310463PMTFLOWCO.CSV
20250401#991310497PMTFLOWCO.CSV
20250324#991310237PMTFLOWCO.CSV
20250327#991410119PMTFLOWCO.CSV
20250401#991410301PMTFLOWCO.CSV
20250323#991310466PMTFLOWCO.CSV
20250326#991310437PMTFLOWCO.CSV
20250402#991310257PMTFLOWCO.CSV
20250327#991310332PMTFLOWCO.CSV
20250323#991310124PMTFLOWCO.CSV
20250329#991310375PMTFLOWCO.CSV
20250319#991410074PMTFLOWCO.CSV
20250328#991310372PMTFLOWCO.CSV
20250329#991310223PMTFLOWCO.CSV
20250323#991310525PMTFLOWCO.CSV
20250325#991310072PMTFLOWCO.CSV
20250327#991310480PMTFLOWCO.CSV
20250323#991310480PMTFLOWCO.CSV
20250324#991310370PMTFLOWCO.CSV
20250330#991404298PMTFLOWCO.CSV
20250329#991310422PMTFLOWCO.CSV
20250328#991310475PMTFLOWCO.CSV
20250331#991310482PMTFLOWCO.CSV
20250323#991310470PMTFLOWCO.CSV
20250322#991310284PMTFLOWCO.CSV
20250329#991310258PMTFLOWCO.CSV
20250319#991410063PMTFLOWCO.CSV
20250322#991310125PMTFLOWCO.CSV
20250329#991310512PMTFLOWCO.CSV
20250402#991310469PMTFLOWCO.CSV
20250321#991310371PMTFLOWCO.CSV
20250328#991410119PMTFLOWCO.CSV
20250325#991310506PMTFLOWCO.CSV
20250330#991310398PMTFLOWCO.CSV
20250401#991310409PMTFLOWCO.CSV
20250319#991310413PMTFLOWCO.CSV
20250326#991410105PMTFLOWCO.CSV
20250322#991310416PMTFLOWCO.CSV
20250327#991310088PMTFLOWCO.CSV
20250324#991310422PMTFLOWCO.CSV
20250329#991310309PMTFLOWCO.CSV
20250327#991310472PMTFLOWCO.CSV
20250321#991310357PMTFLOWCO.CSV
20250320#991410142PMTFLOWCO.CSV
20250401#991310537PMTFLOWCO.CSV
20250319#991310373PMTFLOWCO.CSV
20250327#991310445PMTFLOWCO.CSV
20250319#991310535PMTFLOWCO.CSV
20250326#991310124PMTFLOWCO.CSV
20250327#991310391PMTFLOWCO.CSV
20250319#991310432PMTFLOWCO.CSV
20250325#991410254PMTFLOWCO.CSV
20250327#991310489PMTFLOWCO.CSV
20250320#991310371PMTFLOWCO.CSV
20250324#991310363PMTFLOWCO.CSV
20250320#991410079PMTFLOWCO.CSV
20250324#991404298PMTFLOWCO.CSV
20250327#991310416PMTFLOWCO.CSV
20250331#991303286PMTFLOWCO.CSV
20250330#991310360PMTFLOWCO.CSV
20250402#991404191PMTFLOWCO.CSV
20250329#991310460PMTFLOWCO.CSV
20250326#991310422PMTFLOWCO.CSV
20250402#991404287PMTFLOWCO.CSV
20250326#991310372PMTFLOWCO.CSV
20250327#991310348PMTFLOWCO.CSV
20250401#991310327PMTFLOWCO.CSV
20250322#991310291PMTFLOWCO.CSV
20250321#991310466PMTFLOWCO.CSV
20250401#991404287PMTFLOWCO.CSV
20250402#991303175PMTFLOWCO.CSV
20250331#991310363PMTFLOWCO.CSV
20250330#991310443PMTFLOWCO.CSV
20250330#991310218PMTFLOWCO.CSV
20250328#991310338PMTFLOWCO.CSV
20250321#991310439PMTFLOWCO.CSV
20250322#991310445PMTFLOWCO.CSV
20250319#991310375PMTFLOWCO.CSV
20250324#991310415PMTFLOWCO.CSV
20250321#991310408PMTFLOWCO.CSV
20250328#991310309PMTFLOWCO.CSV
20250327#991310462PMTFLOWCO.CSV
20250322#991310318PMTFLOWCO.CSV
20250329#991310545PMTFLOWCO.CSV
20250323#991310238PMTFLOWCO.CSV
20250330#991310124PMTFLOWCO.CSV
20250330#991310438PMTFLOWCO.CSV
20250321#991310291PMTFLOWCO.CSV
20250325#991310370PMTFLOWCO.CSV
20250319#991310361PMTFLOWCO.CSV
20250330#991310118PMTFLOWCO.CSV
20250326#991310424PMTFLOWCO.CSV
20250321#991310406PMTFLOWCO.CSV
20250322#991310440PMTFLOWCO.CSV
20250319#991404287PMTFLOWCO.CSV
20250326#991310418PMTFLOWCO.CSV
20250325#991310125PMTFLOWCO.CSV
20250325#991310364PMTFLOWCO.CSV
20250324#991310525PMTFLOWCO.CSV
20250324#991310401PMTFLOWCO.CSV
20250320#991310364PMTFLOWCO.CSV
20250320#991310363PMTFLOWCO.CSV
20250322#991310479PMTFLOWCO.CSV
20250326#991303175PMTFLOWCO.CSV
20250328#991310539PMTFLOWCO.CSV
20250328#991310469PMTFLOWCO.CSV
20250401#991310424PMTFLOWCO.CSV
20250330#991410120PMTFLOWCO.CSV
20250330#991310372PMTFLOWCO.CSV
20250323#991310349PMTFLOWCO.CSV
20250329#991310180PMTFLOWCO.CSV
20250322#991310322PMTFLOWCO.CSV
20250327#991310228PMTFLOWCO.CSV
20250326#991310408PMTFLOWCO.CSV
20250331#991310535PMTFLOWCO.CSV
20250322#991310505PMTFLOWCO.CSV
20250323#991404387PMTFLOWCO.CSV
20250323#991310404PMTFLOWCO.CSV
20250323#991404646PMTFLOWCO.CSV
20250320#991310432PMTFLOWCO.CSV
20250330#991310511PMTFLOWCO.CSV
20250331#991310518PMTFLOWCO.CSV
20250322#991310404PMTFLOWCO.CSV
20250327#991310487PMTFLOWCO.CSV
20250322#991310492PMTFLOWCO.CSV
20250329#991310442PMTFLOWCO.CSV
20250319#991310403PMTFLOWCO.CSV
20250327#991404646PMTFLOWCO.CSV
20250324#991310464PMTFLOWCO.CSV
20250327#991310422PMTFLOWCO.CSV
20250319#991310459PMTFLOWCO.CSV
20250402#991310361PMTFLOWCO.CSV
20250319#991310412PMTFLOWCO.CSV
20250323#991410147PMTFLOWCO.CSV
20250402#991404871PMTFLOWCO.CSV
20250326#991310369PMTFLOWCO.CSV
20250331#991310409PMTFLOWCO.CSV
20250330#991310505PMTFLOWCO.CSV
20250328#991310518PMTFLOWCO.CSV
20250324#991404178PMTFLOWCO.CSV
20250329#991310498PMTFLOWCO.CSV
20250323#991310530PMTFLOWCO.CSV
20250326#991410301PMTFLOWCO.CSV
20250323#991310371PMTFLOWCO.CSV
20250331#991310318PMTFLOWCO.CSV
20250402#991310404PMTFLOWCO.CSV
20250402#991310533PMTFLOWCO.CSV
20250322#991410074PMTFLOWCO.CSV
20250330#991310382PMTFLOWCO.CSV
20250326#991310413PMTFLOWCO.CSV
20250319#991410147PMTFLOWCO.CSV
20250326#991310486PMTFLOWCO.CSV
20250401#991404624PMTFLOWCO.CSV
20250325#991410151PMTFLOWCO.CSV
20250325#991310093PMTFLOWCO.CSV
20250331#991310326PMTFLOWCO.CSV
20250329#991410144PMTFLOWCO.CSV
20250326#991310361PMTFLOWCO.CSV
20250327#991410301PMTFLOWCO.CSV
20250322#991310552PMTFLOWCO.CSV
20250321#991310382PMTFLOWCO.CSV
20250402#991310418PMTFLOWCO.CSV
20250321#991310378PMTFLOWCO.CSV
20250401#991310523PMTFLOWCO.CSV
20250326#991410079PMTFLOWCO.CSV
20250325#991310365PMTFLOWCO.CSV
20250330#991310462PMTFLOWCO.CSV
20250319#991410231PMTFLOWCO.CSV
20250325#991310165PMTFLOWCO.CSV
20250329#991310326PMTFLOWCO.CSV
20250319#991410142PMTFLOWCO.CSV
20250329#991310489PMTFLOWCO.CSV
20250330#991310525PMTFLOWCO.CSV
20250330#991310097PMTFLOWCO.CSV
20250326#991310403PMTFLOWCO.CSV
20250320#991310512PMTFLOWCO.CSV
20250325#991310409PMTFLOWCO.CSV
20250324#991310528PMTFLOWCO.CSV
20250328#991404178PMTFLOWCO.CSV
20250325#991310444PMTFLOWCO.CSV
20250324#991310282PMTFLOWCO.CSV
20250323#991310257PMTFLOWCO.CSV
20250324#991410074PMTFLOWCO.CSV
20250330#991310363PMTFLOWCO.CSV
20250324#991310424PMTFLOWCO.CSV
20250328#991310400PMTFLOWCO.CSV
20250401#991310460PMTFLOWCO.CSV
20250325#991310353PMTFLOWCO.CSV
20250319#991310258PMTFLOWCO.CSV
20250322#991310327PMTFLOWCO.CSV
20250324#991303286PMTFLOWCO.CSV
20250330#991304009PMTFLOWCO.CSV
20250322#991404046PMTFLOWCO.CSV
20250323#991410120PMTFLOWCO.CSV
20250326#991310400PMTFLOWCO.CSV
20250331#991404624PMTFLOWCO.CSV
20250402#991310396PMTFLOWCO.CSV
20250324#991310480PMTFLOWCO.CSV
20250324#991310460PMTFLOWCO.CSV
20250328#991310515PMTFLOWCO.CSV
20250323#991310496PMTFLOWCO.CSV
20250322#991410120PMTFLOWCO.CSV
20250320#991410222PMTFLOWCO.CSV
20250326#991310420PMTFLOWCO.CSV
20250401#991310464PMTFLOWCO.CSV
20250402#991310276PMTFLOWCO.CSV
20250324#991404287PMTFLOWCO.CSV
20250402#991310506PMTFLOWCO.CSV
20250331#991310217PMTFLOWCO.CSV
20250331#991310475PMTFLOWCO.CSV
20250326#991310349PMTFLOWCO.CSV
20250329#991310353PMTFLOWCO.CSV
20250324#991410151PMTFLOWCO.CSV
20250326#991310180PMTFLOWCO.CSV
20250326#991310530PMTFLOWCO.CSV
20250319#991404046PMTFLOWCO.CSV
20250401#991310472PMTFLOWCO.CSV
20250330#991310400PMTFLOWCO.CSV
20250325#991310422PMTFLOWCO.CSV
20250324#991304009PMTFLOWCO.CSV
20250324#991310492PMTFLOWCO.CSV
20250402#991310318PMTFLOWCO.CSV
20250331#991310505PMTFLOWCO.CSV
20250328#991310497PMTFLOWCO.CSV
20250329#991310322PMTFLOWCO.CSV
20250320#991310442PMTFLOWCO.CSV
20250329#991310356PMTFLOWCO.CSV
20250402#991310351PMTFLOWCO.CSV
20250324#991310392PMTFLOWCO.CSV
20250323#991310363PMTFLOWCO.CSV
20250321#991410301PMTFLOWCO.CSV
20250325#991310412PMTFLOWCO.CSV
20250320#991310280PMTFLOWCO.CSV
20250323#991310438PMTFLOWCO.CSV
20250328#991310466PMTFLOWCO.CSV
20250402#991410120PMTFLOWCO.CSV
20250328#991310093PMTFLOWCO.CSV
20250327#991310420PMTFLOWCO.CSV
20250331#991410254PMTFLOWCO.CSV
20250323#991310523PMTFLOWCO.CSV
20250328#991303175PMTFLOWCO.CSV
20250325#991310379PMTFLOWCO.CSV
20250330#991410119PMTFLOWCO.CSV
20250320#991310420PMTFLOWCO.CSV
20250329#991310289PMTFLOWCO.CSV
20250327#991404178PMTFLOWCO.CSV
20250402#991404353PMTFLOWCO.CSV
20250321#991310460PMTFLOWCO.CSV
20250327#991310412PMTFLOWCO.CSV
20250401#991310412PMTFLOWCO.CSV
20250321#991310349PMTFLOWCO.CSV
20250319#991310112PMTFLOWCO.CSV
20250401#991310486PMTFLOWCO.CSV
20250327#991303175PMTFLOWCO.CSV
20250329#991310291PMTFLOWCO.CSV
20250328#991310408PMTFLOWCO.CSV
20250329#991310372PMTFLOWCO.CSV
20250324#991310399PMTFLOWCO.CSV
20250401#991310360PMTFLOWCO.CSV
20250324#991310291PMTFLOWCO.CSV
20250327#991310207PMTFLOWCO.CSV
20250330#991310359PMTFLOWCO.CSV
20250319#991310360PMTFLOWCO.CSV
20250325#991310507PMTFLOWCO.CSV
20250402#991310255PMTFLOWCO.CSV
20250329#991310118PMTFLOWCO.CSV
20250321#991310405PMTFLOWCO.CSV
20250401#991310357PMTFLOWCO.CSV
20250322#991310484PMTFLOWCO.CSV
20250328#991310496PMTFLOWCO.CSV
20250329#991310373PMTFLOWCO.CSV
20250320#991410117PMTFLOWCO.CSV
20250329#991310541PMTFLOWCO.CSV
20250402#991310413PMTFLOWCO.CSV
20250322#991310470PMTFLOWCO.CSV
20250329#991310465PMTFLOWCO.CSV
20250329#991310530PMTFLOWCO.CSV
20250325#991310088PMTFLOWCO.CSV
20250328#991310455PMTFLOWCO.CSV
20250331#991310479PMTFLOWCO.CSV
20250402#991404359PMTFLOWCO.CSV
20250330#991310465PMTFLOWCO.CSV
20250330#991310434PMTFLOWCO.CSV
20250324#991310497PMTFLOWCO.CSV
20250402#991410144PMTFLOWCO.CSV
20250325#991310537PMTFLOWCO.CSV
20250321#991310511PMTFLOWCO.CSV
20250319#991310118PMTFLOWCO.CSV
20250323#991310416PMTFLOWCO.CSV
20250325#991310480PMTFLOWCO.CSV
20250319#991404292PMTFLOWCO.CSV
20250329#991310371PMTFLOWCO.CSV
20250328#991310072PMTFLOWCO.CSV
20250321#991310396PMTFLOWCO.CSV
20250320#991310257PMTFLOWCO.CSV
20250324#991310537PMTFLOWCO.CSV
20250325#991310372PMTFLOWCO.CSV
20250330#991310280PMTFLOWCO.CSV
20250322#991310461PMTFLOWCO.CSV
20250320#991310373PMTFLOWCO.CSV
20250330#991310327PMTFLOWCO.CSV
20250323#991310278PMTFLOWCO.CSV
20250320#991310482PMTFLOWCO.CSV
20250328#991310261PMTFLOWCO.CSV
20250324#991404191PMTFLOWCO.CSV
20250322#991310392PMTFLOWCO.CSV
20250322#991310371PMTFLOWCO.CSV
20250330#991310459PMTFLOWCO.CSV
20250328#991310369PMTFLOWCO.CSV
20250329#991310379PMTFLOWCO.CSV
20250331#991310472PMTFLOWCO.CSV
20250329#991310237PMTFLOWCO.CSV
20250324#991410209PMTFLOWCO.CSV
20250318#991404046PMTFLOWCO.CSV
20250329#991310365PMTFLOWCO.CSV
20250325#991310533PMTFLOWCO.CSV
20250402#991303930PMTFLOWCO.CSV
20250331#991310382PMTFLOWCO.CSV
20250323#991310255PMTFLOWCO.CSV
20250329#991310278PMTFLOWCO.CSV
20250322#991310258PMTFLOWCO.CSV
20250321#991310278PMTFLOWCO.CSV
20250321#991410120PMTFLOWCO.CSV
20250402#991310464PMTFLOWCO.CSV
20250321#991410105PMTFLOWCO.CSV
20250330#991310472PMTFLOWCO.CSV
20250326#991310492PMTFLOWCO.CSV
20250319#991310124PMTFLOWCO.CSV
20250327#991310285PMTFLOWCO.CSV
20250401#991404359PMTFLOWCO.CSV
20250329#991310443PMTFLOWCO.CSV
20250330#991310539PMTFLOWCO.CSV
20250319#991310322PMTFLOWCO.CSV
20250319#991310239PMTFLOWCO.CSV
20250402#991304009PMTFLOWCO.CSV
20250402#991310541PMTFLOWCO.CSV
20250330#991310349PMTFLOWCO.CSV
20250328#991410120PMTFLOWCO.CSV
20250326#991310415PMTFLOWCO.CSV
20250401#991310452PMTFLOWCO.CSV
20250401#991310118PMTFLOWCO.CSV
20250328#991310525PMTFLOWCO.CSV
20250328#991310409PMTFLOWCO.CSV
20250326#991310497PMTFLOWCO.CSV
20250324#991310396PMTFLOWCO.CSV
20250319#991310439PMTFLOWCO.CSV
20250322#991310512PMTFLOWCO.CSV
20250325#991310530PMTFLOWCO.CSV
20250329#991310487PMTFLOWCO.CSV
20250322#991310369PMTFLOWCO.CSV
20250327#991310443PMTFLOWCO.CSV
20250321#991310438PMTFLOWCO.CSV
20250320#991310480PMTFLOWCO.CSV
20250402#991410222PMTFLOWCO.CSV
20250330#991310415PMTFLOWCO.CSV
20250326#991310394PMTFLOWCO.CSV
20250321#991310322PMTFLOWCO.CSV
20250319#991304009PMTFLOWCO.CSV
20250329#991310376PMTFLOWCO.CSV
20250323#991310403PMTFLOWCO.CSV
20250321#991310285PMTFLOWCO.CSV
20250325#991310405PMTFLOWCO.CSV
20250331#991310438PMTFLOWCO.CSV
20250321#991410117PMTFLOWCO.CSV
20250327#991304009PMTFLOWCO.CSV
20250322#991310379PMTFLOWCO.CSV
20250331#991310408PMTFLOWCO.CSV
20250324#991310348PMTFLOWCO.CSV
20250328#991310285PMTFLOWCO.CSV
20250328#991310217PMTFLOWCO.CSV
20250320#991310285PMTFLOWCO.CSV
20250327#991310318PMTFLOWCO.CSV
20250321#991310258PMTFLOWCO.CSV
20250402#991404624PMTFLOWCO.CSV
20250327#991310463PMTFLOWCO.CSV
20250319#991310372PMTFLOWCO.CSV
20250326#991310455PMTFLOWCO.CSV
20250329#991310455PMTFLOWCO.CSV
20250402#991310376PMTFLOWCO.CSV
20250325#991310492PMTFLOWCO.CSV
20250402#991310285PMTFLOWCO.CSV
20250330#991310552PMTFLOWCO.CSV
20250324#991310228PMTFLOWCO.CSV
20250321#991310398PMTFLOWCO.CSV
20250324#991310535PMTFLOWCO.CSV
20250327#991410254PMTFLOWCO.CSV
20250320#991310470PMTFLOWCO.CSV
20250331#991410183PMTFLOWCO.CSV
20250331#991310452PMTFLOWCO.CSV
20250321#991310180PMTFLOWCO.CSV
20250323#991310392PMTFLOWCO.CSV
20250401#991410189PMTFLOWCO.CSV
20250323#991310533PMTFLOWCO.CSV
20250327#991310327PMTFLOWCO.CSV
20250329#991310402PMTFLOWCO.CSV
20250319#991310523PMTFLOWCO.CSV
20250319#991310511PMTFLOWCO.CSV
20250319#991310218PMTFLOWCO.CSV
20250326#991310430PMTFLOWCO.CSV
20250321#991310118PMTFLOWCO.CSV
20250323#991310353PMTFLOWCO.CSV
20250320#991310422PMTFLOWCO.CSV
20250319#991410144PMTFLOWCO.CSV
20250326#991310487PMTFLOWCO.CSV
20250324#991310409PMTFLOWCO.CSV
20250319#991404871PMTFLOWCO.CSV
20250329#991310412PMTFLOWCO.CSV
20250324#991310326PMTFLOWCO.CSV
20250325#991404353PMTFLOWCO.CSV
20250330#991310357PMTFLOWCO.CSV
20250322#991310537PMTFLOWCO.CSV
20250319#991310223PMTFLOWCO.CSV
20250327#991310455PMTFLOWCO.CSV
20250330#991404359PMTFLOWCO.CSV
20250330#991310479PMTFLOWCO.CSV
20250331#991310238PMTFLOWCO.CSV
20250331#991310401PMTFLOWCO.CSV
20250401#991310125PMTFLOWCO.CSV
20250324#991310327PMTFLOWCO.CSV
20250321#991310097PMTFLOWCO.CSV
20250320#991310506PMTFLOWCO.CSV
20250321#991310392PMTFLOWCO.CSV
20250331#991310372PMTFLOWCO.CSV
20250320#991310455PMTFLOWCO.CSV
20250323#991310373PMTFLOWCO.CSV
20250327#991310222PMTFLOWCO.CSV
20250324#991310369PMTFLOWCO.CSV
20250328#991310552PMTFLOWCO.CSV
20250320#991310391PMTFLOWCO.CSV
20250328#991310537PMTFLOWCO.CSV
20250324#991310445PMTFLOWCO.CSV
20250328#991310452PMTFLOWCO.CSV
20250402#991310202PMTFLOWCO.CSV
20250325#991303175PMTFLOWCO.CSV
20250323#991310239PMTFLOWCO.CSV
20250331#991310375PMTFLOWCO.CSV
20250401#991310088PMTFLOWCO.CSV
20250321#991310537PMTFLOWCO.CSV
20250401#991310072PMTFLOWCO.CSV
20250331#991310396PMTFLOWCO.CSV
20250326#991310462PMTFLOWCO.CSV
20250327#991310459PMTFLOWCO.CSV
20250401#991310539PMTFLOWCO.CSV
20250328#991310401PMTFLOWCO.CSV
20250328#991404353PMTFLOWCO.CSV
20250401#991310364PMTFLOWCO.CSV
20250324#991310512PMTFLOWCO.CSV
20250322#991310497PMTFLOWCO.CSV
20250323#991310415PMTFLOWCO.CSV
20250323#991310369PMTFLOWCO.CSV
20250320#991310382PMTFLOWCO.CSV
20250402#991310475PMTFLOWCO.CSV
20250322#991410186PMTFLOWCO.CSV
20250402#991310420PMTFLOWCO.CSV
20250329#991310552PMTFLOWCO.CSV
20250331#991310257PMTFLOWCO.CSV
20250321#991410223PMTFLOWCO.CSV
20250323#991404191PMTFLOWCO.CSV
20250329#991310433PMTFLOWCO.CSV
20250324#991310309PMTFLOWCO.CSV
20250323#991310420PMTFLOWCO.CSV
20250331#991310455PMTFLOWCO.CSV
20250320#991310444PMTFLOWCO.CSV
20250321#991410209PMTFLOWCO.CSV
20250325#991404046PMTFLOWCO.CSV
20250324#991310463PMTFLOWCO.CSV
20250331#991310289PMTFLOWCO.CSV
20250323#991310217PMTFLOWCO.CSV
20250321#991310261PMTFLOWCO.CSV
20250330#991310348PMTFLOWCO.CSV
20250320#991404624PMTFLOWCO.CSV
20250322#991310372PMTFLOWCO.CSV
20250328#991310399PMTFLOWCO.CSV
20250319#991310285PMTFLOWCO.CSV
20250330#991310498PMTFLOWCO.CSV
20250322#991310282PMTFLOWCO.CSV
20250327#991310255PMTFLOWCO.CSV
20250321#991310372PMTFLOWCO.CSV
20250331#991310445PMTFLOWCO.CSV
20250329#991310239PMTFLOWCO.CSV
20250329#991310391PMTFLOWCO.CSV
20250320#991310406PMTFLOWCO.CSV
20250326#991310489PMTFLOWCO.CSV
20250330#991310282PMTFLOWCO.CSV
20250322#991310501PMTFLOWCO.CSV
20250320#991310487PMTFLOWCO.CSV
20250322#991310400PMTFLOWCO.CSV
20250331#991404178PMTFLOWCO.CSV
20250325#991410223PMTFLOWCO.CSV
20250326#991310359PMTFLOWCO.CSV
20250322#991310538PMTFLOWCO.CSV
20250320#991310291PMTFLOWCO.CSV
20250402#991310398PMTFLOWCO.CSV
20250323#991410223PMTFLOWCO.CSV
20250331#991310465PMTFLOWCO.CSV
20250328#991410254PMTFLOWCO.CSV
20250321#991310072PMTFLOWCO.CSV
20250401#991310177PMTFLOWCO.CSV
20250331#991310180PMTFLOWCO.CSV
20250330#991310453PMTFLOWCO.CSV
20250325#991310322PMTFLOWCO.CSV
20250322#991310452PMTFLOWCO.CSV
20250324#991310465PMTFLOWCO.CSV
20250322#991404387PMTFLOWCO.CSV
20250323#991310361PMTFLOWCO.CSV
20250325#991303286PMTFLOWCO.CSV
20250402#991310338PMTFLOWCO.CSV
20250327#991310424PMTFLOWCO.CSV
20250330#991310222PMTFLOWCO.CSV
20250322#991404624PMTFLOWCO.CSV
20250401#991310502PMTFLOWCO.CSV
20250328#991310502PMTFLOWCO.CSV
20250319#991310327PMTFLOWCO.CSV
20250331#991410074PMTFLOWCO.CSV
20250322#991310353PMTFLOWCO.CSV
20250323#991310237PMTFLOWCO.CSV
20250326#991310239PMTFLOWCO.CSV
20250323#991303175PMTFLOWCO.CSV
20250402#991410147PMTFLOWCO.CSV
20250401#991310280PMTFLOWCO.CSV
20250401#991310351PMTFLOWCO.CSV
20250327#991310361PMTFLOWCO.CSV
20250401#991410186PMTFLOWCO.CSV
20250324#991310072PMTFLOWCO.CSV
20250326#991310284PMTFLOWCO.CSV
20250323#991310505PMTFLOWCO.CSV
20250401#991310284PMTFLOWCO.CSV
20250319#991310364PMTFLOWCO.CSV
20250331#991310394PMTFLOWCO.CSV
20250327#991310497PMTFLOWCO.CSV
20250331#991310459PMTFLOWCO.CSV
20250323#991303286PMTFLOWCO.CSV
20250322#991310326PMTFLOWCO.CSV
20250330#991310510PMTFLOWCO.CSV
20250401#991303175PMTFLOWCO.CSV
20250328#991310396PMTFLOWCO.CSV
20250325#991310505PMTFLOWCO.CSV
20250402#991310392PMTFLOWCO.CSV
20250326#991310125PMTFLOWCO.CSV
20250328#991310353PMTFLOWCO.CSV
20250321#991310165PMTFLOWCO.CSV
20250330#991310501PMTFLOWCO.CSV
20250329#991310416PMTFLOWCO.CSV
20250323#991410183PMTFLOWCO.CSV
20250320#991410147PMTFLOWCO.CSV
20250319#991310391PMTFLOWCO.CSV
20250326#991310276PMTFLOWCO.CSV
20250331#991310464PMTFLOWCO.CSV
20250402#991310378PMTFLOWCO.CSV
20250401#991310202PMTFLOWCO.CSV
20250320#991310353PMTFLOWCO.CSV
20250321#991310217PMTFLOWCO.CSV
20250329#991310357PMTFLOWCO.CSV
20250328#991310501PMTFLOWCO.CSV
20250401#991310533PMTFLOWCO.CSV
20250402#991410209PMTFLOWCO.CSV
20250401#991310459PMTFLOWCO.CSV
20250320#991310258PMTFLOWCO.CSV
20250401#991310422PMTFLOWCO.CSV
20250324#991310539PMTFLOWCO.CSV
20250323#991404298PMTFLOWCO.CSV
20250325#991410183PMTFLOWCO.CSV
20250331#991310373PMTFLOWCO.CSV
20250325#991310453PMTFLOWCO.CSV
20250401#991410063PMTFLOWCO.CSV
20250330#991310535PMTFLOWCO.CSV
20250321#991310359PMTFLOWCO.CSV
20250321#991310379PMTFLOWCO.CSV
20250327#991310538PMTFLOWCO.CSV
20250321#991404646PMTFLOWCO.CSV
20250331#991404353PMTFLOWCO.CSV
20250319#991303930PMTFLOWCO.CSV
20250327#991310541PMTFLOWCO.CSV
20250327#991404624PMTFLOWCO.CSV
20250401#991310461PMTFLOWCO.CSV
20250320#991310465PMTFLOWCO.CSV
20250321#991310403PMTFLOWCO.CSV
20250320#991310479PMTFLOWCO.CSV
20250323#991310440PMTFLOWCO.CSV
20250329#991404646PMTFLOWCO.CSV
20250330#991310470PMTFLOWCO.CSV
20250320#991310507PMTFLOWCO.CSV
20250324#991310416PMTFLOWCO.CSV
20250322#991404298PMTFLOWCO.CSV
20250326#991303286PMTFLOWCO.CSV
20250323#991310460PMTFLOWCO.CSV
20250401#991310463PMTFLOWCO.CSV
20250331#991410147PMTFLOWCO.CSV
20250319#991310460PMTFLOWCO.CSV
20250321#991310523PMTFLOWCO.CSV
20250322#991310506PMTFLOWCO.CSV
20250323#991310424PMTFLOWCO.CSV
20250331#991310327PMTFLOWCO.CSV
20250327#991310353PMTFLOWCO.CSV
20250331#991310443PMTFLOWCO.CSV
20250326#991303930PMTFLOWCO.CSV
20250320#991410151PMTFLOWCO.CSV
20250327#991310430PMTFLOWCO.CSV
20250324#991310177PMTFLOWCO.CSV
20250322#991410223PMTFLOWCO.CSV
20250320#991310396PMTFLOWCO.CSV
20250402#991310309PMTFLOWCO.CSV
20250322#991310418PMTFLOWCO.CSV
20250324#991310479PMTFLOWCO.CSV
20250319#991410254PMTFLOWCO.CSV
20250324#991310444PMTFLOWCO.CSV
20250401#991310511PMTFLOWCO.CSV
20250320#991310088PMTFLOWCO.CSV
20250327#991310289PMTFLOWCO.CSV
20250331#991310546PMTFLOWCO.CSV
20250328#991310165PMTFLOWCO.CSV
20250401#991410142PMTFLOWCO.CSV
20250328#991310403PMTFLOWCO.CSV
20250325#991310153PMTFLOWCO.CSV
20250319#991404184PMTFLOWCO.CSV
20250327#991310372PMTFLOWCO.CSV
20250325#991310487PMTFLOWCO.CSV
20250329#991404624PMTFLOWCO.CSV
20250322#991310464PMTFLOWCO.CSV
20250328#991310464PMTFLOWCO.CSV
20250326#991310439PMTFLOWCO.CSV
20250401#991304009PMTFLOWCO.CSV
20250322#991304009PMTFLOWCO.CSV
20250320#991310505PMTFLOWCO.CSV
20250319#991310455PMTFLOWCO.CSV
20250325#991310207PMTFLOWCO.CSV
20250326#991310223PMTFLOWCO.CSV
20250402#991310405PMTFLOWCO.CSV
20250321#991310215PMTFLOWCO.CSV
20250325#991310338PMTFLOWCO.CSV
20250326#991404298PMTFLOWCO.CSV
20250319#991310463PMTFLOWCO.CSV
20250328#991310442PMTFLOWCO.CSV
20250401#991310443PMTFLOWCO.CSV
20250320#991410063PMTFLOWCO.CSV
20250324#991404292PMTFLOWCO.CSV
20250401#991310405PMTFLOWCO.CSV
20250321#991310482PMTFLOWCO.CSV
20250331#991310507PMTFLOWCO.CSV
20250329#991310420PMTFLOWCO.CSV
20250323#991310439PMTFLOWCO.CSV
20250322#991410254PMTFLOWCO.CSV
20250402#991410119PMTFLOWCO.CSV
20250326#991310525PMTFLOWCO.CSV
20250326#991310365PMTFLOWCO.CSV
20250326#991310391PMTFLOWCO.CSV
20250324#991310439PMTFLOWCO.CSV
20250323#991310357PMTFLOWCO.CSV
20250324#991310461PMTFLOWCO.CSV
20250325#991310398PMTFLOWCO.CSV
20250328#991410186PMTFLOWCO.CSV
20250320#991310409PMTFLOWCO.CSV
20250323#991310538PMTFLOWCO.CSV
20250326#991310510PMTFLOWCO.CSV
20250319#991410079PMTFLOWCO.CSV
20250330#991310278PMTFLOWCO.CSV
20250319#991410301PMTFLOWCO.CSV
20250321#991310222PMTFLOWCO.CSV
20250319#991310309PMTFLOWCO.CSV
20250401#991310378PMTFLOWCO.CSV
20250319#991404298PMTFLOWCO.CSV
20250327#991310465PMTFLOWCO.CSV
20250320#991310372PMTFLOWCO.CSV
20250326#991310278PMTFLOWCO.CSV
20250323#991310382PMTFLOWCO.CSV
20250320#991310276PMTFLOWCO.CSV
20250322#991410222PMTFLOWCO.CSV
20250402#991310510PMTFLOWCO.CSV
20250330#991310424PMTFLOWCO.CSV
20250402#991410079PMTFLOWCO.CSV
20250401#991310507PMTFLOWCO.CSV
20250331#991310506PMTFLOWCO.CSV
20250402#991410142PMTFLOWCO.CSV
20250330#991310502PMTFLOWCO.CSV
20250328#991310489PMTFLOWCO.CSV
20250324#991404646PMTFLOWCO.CSV
20250319#991310278PMTFLOWCO.CSV
20250327#991310280PMTFLOWCO.CSV
20250329#991310382PMTFLOWCO.CSV
20250401#991410231PMTFLOWCO.CSV
20250402#991310238PMTFLOWCO.CSV
20250326#991310338PMTFLOWCO.CSV
20250321#991310400PMTFLOWCO.CSV
20250402#991310484PMTFLOWCO.CSV
20250402#991310284PMTFLOWCO.CSV
20250330#991404046PMTFLOWCO.CSV
20250325#991303930PMTFLOWCO.CSV
20250330#991410074PMTFLOWCO.CSV
20250401#991404298PMTFLOWCO.CSV
20250320#991404191PMTFLOWCO.CSV
20250331#991310379PMTFLOWCO.CSV
20250401#991310439PMTFLOWCO.CSV
20250326#991310093PMTFLOWCO.CSV
20250402#991310439PMTFLOWCO.CSV
20250329#991310525PMTFLOWCO.CSV
20250320#991310416PMTFLOWCO.CSV
20250329#991410117PMTFLOWCO.CSV
20250321#991310412PMTFLOWCO.CSV
20250320#991404387PMTFLOWCO.CSV
20250324#991404359PMTFLOWCO.CSV
20250320#991310361PMTFLOWCO.CSV
20250402#991310489PMTFLOWCO.CSV
20250401#991310391PMTFLOWCO.CSV
20250324#991310518PMTFLOWCO.CSV
20250331#991310533PMTFLOWCO.CSV
20250331#991310413PMTFLOWCO.CSV
20250327#991310433PMTFLOWCO.CSV
20250401#991310465PMTFLOWCO.CSV
20250327#991310180PMTFLOWCO.CSV
20250331#991310466PMTFLOWCO.CSV
20250331#991310222PMTFLOWCO.CSV
20250402#991310406PMTFLOWCO.CSV
20250319#991310525PMTFLOWCO.CSV
20250324#991310289PMTFLOWCO.CSV
20250321#991310424PMTFLOWCO.CSV
20250321#991410151PMTFLOWCO.CSV
20250328#991310365PMTFLOWCO.CSV
20250328#991310538PMTFLOWCO.CSV
20250401#991310237PMTFLOWCO.CSV
20250320#991310510PMTFLOWCO.CSV
20250323#991304009PMTFLOWCO.CSV
20250324#991404184PMTFLOWCO.CSV
20250320#991310097PMTFLOWCO.CSV
20250329#991404359PMTFLOWCO.CSV
20250321#991310455PMTFLOWCO.CSV
20250320#991310460PMTFLOWCO.CSV
20250325#991310326PMTFLOWCO.CSV
20250402#991310443PMTFLOWCO.CSV
20250401#991310398PMTFLOWCO.CSV
20250331#991310285PMTFLOWCO.CSV
20250329#991310363PMTFLOWCO.CSV
20250320#991310223PMTFLOWCO.CSV
20250321#991310506PMTFLOWCO.CSV
20250330#991310455PMTFLOWCO.CSV
20250331#991410186PMTFLOWCO.CSV
20250401#991310432PMTFLOWCO.CSV
20250319#991310416PMTFLOWCO.CSV
20250401#991310261PMTFLOWCO.CSV
20250328#991310222PMTFLOWCO.CSV
20250327#991310444PMTFLOWCO.CSV
20250329#991310424PMTFLOWCO.CSV
20250402#991310124PMTFLOWCO.CSV
20250321#991410063PMTFLOWCO.CSV
20250321#991310416PMTFLOWCO.CSV
20250323#991310437PMTFLOWCO.CSV
20250331#991310430PMTFLOWCO.CSV
20250322#991310401PMTFLOWCO.CSV
20250401#991410254PMTFLOWCO.CSV
20250323#991404287PMTFLOWCO.CSV
20250331#991310424PMTFLOWCO.CSV
20250330#991310394PMTFLOWCO.CSV
20250401#991310400PMTFLOWCO.CSV
20250322#991303175PMTFLOWCO.CSV
20250325#991310430PMTFLOWCO.CSV
20250319#991310088PMTFLOWCO.CSV
20250329#991310361PMTFLOWCO.CSV
20250325#991310289PMTFLOWCO.CSV
20250321#991310505PMTFLOWCO.CSV
20250328#991410117PMTFLOWCO.CSV
20250324#991310412PMTFLOWCO.CSV
20250327#991310360PMTFLOWCO.CSV
20250328#991310238PMTFLOWCO.CSV
20250321#991310518PMTFLOWCO.CSV
20250328#991310422PMTFLOWCO.CSV
20250331#991310487PMTFLOWCO.CSV
20250328#991310505PMTFLOWCO.CSV
20250320#991310515PMTFLOWCO.CSV
20250329#991310464PMTFLOWCO.CSV
20250325#991310470PMTFLOWCO.CSV
20250324#991310379PMTFLOWCO.CSV
20250328#991404646PMTFLOWCO.CSV
20250326#991310215PMTFLOWCO.CSV
20250320#991310459PMTFLOWCO.CSV
20250401#991310552PMTFLOWCO.CSV
20250324#991310489PMTFLOWCO.CSV
20250319#991310510PMTFLOWCO.CSV
20250329#991310394PMTFLOWCO.CSV
20250329#991310396PMTFLOWCO.CSV
20250327#991310415PMTFLOWCO.CSV
20250330#991310545PMTFLOWCO.CSV
20250319#991404646PMTFLOWCO.CSV
20250329#991303286PMTFLOWCO.CSV
20250324#991310357PMTFLOWCO.CSV
20250401#991310445PMTFLOWCO.CSV
20250402#991404178PMTFLOWCO.CSV
20250329#991310444PMTFLOWCO.CSV
20250327#991410186PMTFLOWCO.CSV
20250327#991410105PMTFLOWCO.CSV
20250322#991310460PMTFLOWCO.CSV
20250331#991310125PMTFLOWCO.CSV
20250324#991310255PMTFLOWCO.CSV
20250325#991404292PMTFLOWCO.CSV
20250321#991410189PMTFLOWCO.CSV
20250322#991404191PMTFLOWCO.CSV
20250401#991310437PMTFLOWCO.CSV
20250328#991310239PMTFLOWCO.CSV
20250319#991310282PMTFLOWCO.CSV
20250328#991310327PMTFLOWCO.CSV
20250331#991310088PMTFLOWCO.CSV
20250324#991310332PMTFLOWCO.CSV
20250322#991310228PMTFLOWCO.CSV
20250329#991310492PMTFLOWCO.CSV
20250401#991310215PMTFLOWCO.CSV
20250330#991404353PMTFLOWCO.CSV
20250331#991310523PMTFLOWCO.CSV
20250319#991310257PMTFLOWCO.CSV
20250323#991310510PMTFLOWCO.CSV
20250323#991310258PMTFLOWCO.CSV
20250330#991310482PMTFLOWCO.CSV
20250328#991310541PMTFLOWCO.CSV
20250330#991410209PMTFLOWCO.CSV
20250321#991310472PMTFLOWCO.CSV
20250324#991310501PMTFLOWCO.CSV
20250329#991310215PMTFLOWCO.CSV
20250319#991310348PMTFLOWCO.CSV
20250326#991310373PMTFLOWCO.CSV
20250328#991310125PMTFLOWCO.CSV
20250321#991310238PMTFLOWCO.CSV
20250327#991410063PMTFLOWCO.CSV
20250324#991310257PMTFLOWCO.CSV
20250402#991310422PMTFLOWCO.CSV
20250327#991410183PMTFLOWCO.CSV
20250319#991310202PMTFLOWCO.CSV
20250327#991310257PMTFLOWCO.CSV
20250325#991310545PMTFLOWCO.CSV
20250330#991404292PMTFLOWCO.CSV
20250324#991310276PMTFLOWCO.CSV
20250328#991410063PMTFLOWCO.CSV
20250322#991310398PMTFLOWCO.CSV
20250325#991310510PMTFLOWCO.CSV
20250329#991310418PMTFLOWCO.CSV
20250402#991310359PMTFLOWCO.CSV
20250322#991310424PMTFLOWCO.CSV
20250318#991310422PMTFLOWCO.CSV
20250323#991310430PMTFLOWCO.CSV
20250328#991310415PMTFLOWCO.CSV
20250328#991310180PMTFLOWCO.CSV
20250327#991310484PMTFLOWCO.CSV
20250322#991310237PMTFLOWCO.CSV
20250326#991310461PMTFLOWCO.CSV
20250321#991310338PMTFLOWCO.CSV
20250319#991404232PMTFLOWCO.CSV
20250401#991310416PMTFLOWCO.CSV
20250326#991310217PMTFLOWCO.CSV
20250322#991310486PMTFLOWCO.CSV
20250319#991310276PMTFLOWCO.CSV
20250328#991310462PMTFLOWCO.CSV
20250320#991310398PMTFLOWCO.CSV
20250324#991310373PMTFLOWCO.CSV
20250325#991310466PMTFLOWCO.CSV
20250323#991310177PMTFLOWCO.CSV
20250327#991310510PMTFLOWCO.CSV
20250323#991310461PMTFLOWCO.CSV
20250402#991310399PMTFLOWCO.CSV
20250328#991310445PMTFLOWCO.CSV
20250328#991410183PMTFLOWCO.CSV
20250402#991310430PMTFLOWCO.CSV
20250328#991310370PMTFLOWCO.CSV
20250320#991404184PMTFLOWCO.CSV
20250402#991310433PMTFLOWCO.CSV
20250328#991310153PMTFLOWCO.CSV
20250326#991310398PMTFLOWCO.CSV
20250327#991310326PMTFLOWCO.CSV
20250319#991310215PMTFLOWCO.CSV
20250322#991310409PMTFLOWCO.CSV
20250331#991310432PMTFLOWCO.CSV
20250330#991310422PMTFLOWCO.CSV
20250321#991310475PMTFLOWCO.CSV
20250325#991310392PMTFLOWCO.CSV
20250319#991310424PMTFLOWCO.CSV
20250401#991404178PMTFLOWCO.CSV
20250325#991404298PMTFLOWCO.CSV
20250320#991404178PMTFLOWCO.CSV
20250322#991410119PMTFLOWCO.CSV
20250328#991310492PMTFLOWCO.CSV
20250331#991410209PMTFLOWCO.CSV
20250322#991310403PMTFLOWCO.CSV
20250319#991310430PMTFLOWCO.CSV
20250319#991310480PMTFLOWCO.CSV
20250331#991404184PMTFLOWCO.CSV
20250329#991410063PMTFLOWCO.CSV
20250329#991310469PMTFLOWCO.CSV
20250402#991310364PMTFLOWCO.CSV
20250320#991310238PMTFLOWCO.CSV
20250330#991310318PMTFLOWCO.CSV
20250323#991310364PMTFLOWCO.CSV
20250329#991404232PMTFLOWCO.CSV
20250320#991310511PMTFLOWCO.CSV
20250319#991410105PMTFLOWCO.CSV
20250319#991310404PMTFLOWCO.CSV
20250326#991310399PMTFLOWCO.CSV
20250328#991410189PMTFLOWCO.CSV
20250324#991310530PMTFLOWCO.CSV
20250328#991310392PMTFLOWCO.CSV
20250328#991410074PMTFLOWCO.CSV
20250323#991310497PMTFLOWCO.CSV
20250330#991410063PMTFLOWCO.CSV
20250327#991310237PMTFLOWCO.CSV
20250330#991310512PMTFLOWCO.CSV
20250324#991404308PMTFLOWCO.CSV
20250323#991404308PMTFLOWCO.CSV
20250326#991310460PMTFLOWCO.CSV
20250326#991410119PMTFLOWCO.CSV
20250329#991310349PMTFLOWCO.CSV
20250321#991310309PMTFLOWCO.CSV
20250324#991310469PMTFLOWCO.CSV
20250330#991410117PMTFLOWCO.CSV
20250319#991310497PMTFLOWCO.CSV
20250319#991404191PMTFLOWCO.CSV
20250324#991310455PMTFLOWCO.CSV
20250328#991404359PMTFLOWCO.CSV
20250319#991310546PMTFLOWCO.CSV
20250401#991310469PMTFLOWCO.CSV
20250324#991310541PMTFLOWCO.CSV
20250326#991310258PMTFLOWCO.CSV
20250323#991310506PMTFLOWCO.CSV
20250328#991304009PMTFLOWCO.CSV
20250322#991310394PMTFLOWCO.CSV
20250327#991310309PMTFLOWCO.CSV
20250401#991310433PMTFLOWCO.CSV
20250321#991310401PMTFLOWCO.CSV
20250323#991410144PMTFLOWCO.CSV
20250321#991310280PMTFLOWCO.CSV
20250331#991310118PMTFLOWCO.CSV
20250324#991310498PMTFLOWCO.CSV
20250322#991310261PMTFLOWCO.CSV
20250321#991310284PMTFLOWCO.CSV
20250323#991310125PMTFLOWCO.CSV
20250330#991310361PMTFLOWCO.CSV
20250326#991310470PMTFLOWCO.CSV
20250323#991310511PMTFLOWCO.CSV
20250320#991310356PMTFLOWCO.CSV
20250329#991410222PMTFLOWCO.CSV
20250321#991310239PMTFLOWCO.CSV
20250326#991310291PMTFLOWCO.CSV
20250324#991410079PMTFLOWCO.CSV
20250324#991404624PMTFLOWCO.CSV
20250321#991310437PMTFLOWCO.CSV
20250320#991310202PMTFLOWCO.CSV
20250325#991310376PMTFLOWCO.CSV
20250320#991310217PMTFLOWCO.CSV
20250321#991310496PMTFLOWCO.CSV
20250321#991404387PMTFLOWCO.CSV
20250323#991310502PMTFLOWCO.CSV
20250327#991310401PMTFLOWCO.CSV
20250321#991310376PMTFLOWCO.CSV
20250327#991310453PMTFLOWCO.CSV
20250401#991310545PMTFLOWCO.CSV
20250322#991310439PMTFLOWCO.CSV
20250327#991303286PMTFLOWCO.CSV
20250328#991410147PMTFLOWCO.CSV
20250319#991310518PMTFLOWCO.CSV
20250330#991310461PMTFLOWCO.CSV
20250328#991303286PMTFLOWCO.CSV
20250324#991310482PMTFLOWCO.CSV
20250325#991410301PMTFLOWCO.CSV
20250324#991310400PMTFLOWCO.CSV
20250329#991303175PMTFLOWCO.CSV
20250323#991310464PMTFLOWCO.CSV
20250322#991410151PMTFLOWCO.CSV
20250326#991310552PMTFLOWCO.CSV
20250325#991310461PMTFLOWCO.CSV
20250320#991404232PMTFLOWCO.CSV
20250321#991310373PMTFLOWCO.CSV
20250324#991310472PMTFLOWCO.CSV
20250328#991310535PMTFLOWCO.CSV
20250319#991310338PMTFLOWCO.CSV
20250325#991310552PMTFLOWCO.CSV
20250320#991310523PMTFLOWCO.CSV
20250325#991310360PMTFLOWCO.CSV
20250329#991310408PMTFLOWCO.CSV
20250327#991310238PMTFLOWCO.CSV
20250326#991310360PMTFLOWCO.CSV
20250322#991310413PMTFLOWCO.CSV
20250329#991410151PMTFLOWCO.CSV
20250320#991310497PMTFLOWCO.CSV
20250320#991310498PMTFLOWCO.CSV
20250327#991310552PMTFLOWCO.CSV
20250321#991310459PMTFLOWCO.CSV
20250401#991310257PMTFLOWCO.CSV
20250324#991310406PMTFLOWCO.CSV
20250319#991310097PMTFLOWCO.CSV
20250325#991310438PMTFLOWCO.CSV
20250331#991310416PMTFLOWCO.CSV
20250319#991310487PMTFLOWCO.CSV
20250325#991310237PMTFLOWCO.CSV
20250323#991310072PMTFLOWCO.CSV
20250325#991310432PMTFLOWCO.CSV
20250330#991410223PMTFLOWCO.CSV
20250402#991310502PMTFLOWCO.CSV
20250402#991310400PMTFLOWCO.CSV
20250324#991310442PMTFLOWCO.CSV
20250322#991310097PMTFLOWCO.CSV
20250402#991310538PMTFLOWCO.CSV
20250322#991310472PMTFLOWCO.CSV
20250322#991310437PMTFLOWCO.CSV
20250319#991310093PMTFLOWCO.CSV
20250401#991310124PMTFLOWCO.CSV
20250324#991310545PMTFLOWCO.CSV
20250322#991310088PMTFLOWCO.CSV
20250324#991310322PMTFLOWCO.CSV
20250331#991404287PMTFLOWCO.CSV
20250330#991303175PMTFLOWCO.CSV
20250322#991310406PMTFLOWCO.CSV
20250401#991410105PMTFLOWCO.CSV
20250320#991404292PMTFLOWCO.CSV
20250401#991310370PMTFLOWCO.CSV
20250324#991310486PMTFLOWCO.CSV
20250325#991310348PMTFLOWCO.CSV
20250401#991310505PMTFLOWCO.CSV
20250402#991310497PMTFLOWCO.CSV
20250401#991310239PMTFLOWCO.CSV
20250328#991310373PMTFLOWCO.CSV
20250322#991310487PMTFLOWCO.CSV
20250322#991310402PMTFLOWCO.CSV
20250320#991310322PMTFLOWCO.CSV
20250329#991404292PMTFLOWCO.CSV
20250328#991404191PMTFLOWCO.CSV
20250320#991310518PMTFLOWCO.CSV
20250321#991310125PMTFLOWCO.CSV
20250320#991310535PMTFLOWCO.CSV
20250327#991310437PMTFLOWCO.CSV
20250324#991310285PMTFLOWCO.CSV
20250328#991310359PMTFLOWCO.CSV
20250401#991310218PMTFLOWCO.CSV
20250401#991410151PMTFLOWCO.CSV
20250325#991310442PMTFLOWCO.CSV
20250320#991310525PMTFLOWCO.CSV
20250325#991310443PMTFLOWCO.CSV
20250328#991310097PMTFLOWCO.CSV
20250401#991404308PMTFLOWCO.CSV
20250329#991310153PMTFLOWCO.CSV
20250320#991310430PMTFLOWCO.CSV
20250327#991310399PMTFLOWCO.CSV
20250325#991404624PMTFLOWCO.CSV
20250322#991310432PMTFLOWCO.CSV
20250401#991310349PMTFLOWCO.CSV
20250320#991310125PMTFLOWCO.CSV
20250326#991310506PMTFLOWCO.CSV
20250328#991310545PMTFLOWCO.CSV
20250330#991310420PMTFLOWCO.CSV
20250331#991310391PMTFLOWCO.CSV
20250327#991310530PMTFLOWCO.CSV
20250326#991310072PMTFLOWCO.CSV
20250328#991310434PMTFLOWCO.CSV
20250326#991310537PMTFLOWCO.CSV
20250329#991310276PMTFLOWCO.CSV
20250331#991410119PMTFLOWCO.CSV
20250331#991410120PMTFLOWCO.CSV
20250327#991310498PMTFLOWCO.CSV
20250323#991310442PMTFLOWCO.CSV
20250326#991310433PMTFLOWCO.CSV
20250327#991310506PMTFLOWCO.CSV
20250326#991410223PMTFLOWCO.CSV
20250329#991310453PMTFLOWCO.CSV
20250328#991310438PMTFLOWCO.CSV
20250321#991310484PMTFLOWCO.CSV
20250402#991310511PMTFLOWCO.CSV
20250402#991310445PMTFLOWCO.CSV
20250330#991310533PMTFLOWCO.CSV
20250331#991310484PMTFLOWCO.CSV
20250325#991410119PMTFLOWCO.CSV
20250401#991310528PMTFLOWCO.CSV
20250323#991310396PMTFLOWCO.CSV
20250322#991310523PMTFLOWCO.CSV
20250325#991310177PMTFLOWCO.CSV
20250401#991303286PMTFLOWCO.CSV
20250331#991410301PMTFLOWCO.CSV
20250330#991310261PMTFLOWCO.CSV
20250402#991310461PMTFLOWCO.CSV
20250324#991310239PMTFLOWCO.CSV
20250319#991310496PMTFLOWCO.CSV
20250327#991310124PMTFLOWCO.CSV
20250325#991410079PMTFLOWCO.CSV
20250401#991404191PMTFLOWCO.CSV
20250326#991410063PMTFLOWCO.CSV
20250327#991404359PMTFLOWCO.CSV
20250323#991310412PMTFLOWCO.CSV
20250401#991310535PMTFLOWCO.CSV
20250323#991310432PMTFLOWCO.CSV
20250327#991310097PMTFLOWCO.CSV
20250323#991310434PMTFLOWCO.CSV
20250324#991310356PMTFLOWCO.CSV
20250328#991310332PMTFLOWCO.CSV
20250331#991310398PMTFLOWCO.CSV
20250331#991310349PMTFLOWCO.CSV
20250401#991310371PMTFLOWCO.CSV
20250322#991310455PMTFLOWCO.CSV
20250330#991310437PMTFLOWCO.CSV
20250329#991310217PMTFLOWCO.CSV
20250322#991310378PMTFLOWCO.CSV
20250322#991310364PMTFLOWCO.CSV
20250329#991310445PMTFLOWCO.CSV
20250329#991310403PMTFLOWCO.CSV
20250325#991410144PMTFLOWCO.CSV
20250325#991310459PMTFLOWCO.CSV
20250323#991310492PMTFLOWCO.CSV
20250328#991310322PMTFLOWCO.CSV
20250324#991310278PMTFLOWCO.CSV
20250331#991303930PMTFLOWCO.CSV
20250319#991310399PMTFLOWCO.CSV
20250402#991310370PMTFLOWCO.CSV
20250331#991310480PMTFLOWCO.CSV
20250331#991310215PMTFLOWCO.CSV
20250327#991310469PMTFLOWCO.CSV
20250321#991303930PMTFLOWCO.CSV
20250328#991310406PMTFLOWCO.CSV
20250331#991310406PMTFLOWCO.CSV
20250326#991410183PMTFLOWCO.CSV
20250323#991310408PMTFLOWCO.CSV
20250325#991310528PMTFLOWCO.CSV
20250401#991310353PMTFLOWCO.CSV
20250321#991310545PMTFLOWCO.CSV
20250320#991410189PMTFLOWCO.CSV
20250328#991310289PMTFLOWCO.CSV
20250402#991310327PMTFLOWCO.CSV
20250322#991404646PMTFLOWCO.CSV
20250326#991310364PMTFLOWCO.CSV
20250402#991310496PMTFLOWCO.CSV
20250321#991310402PMTFLOWCO.CSV
20250401#991310453PMTFLOWCO.CSV
20250323#991310402PMTFLOWCO.CSV
20250324#991310202PMTFLOWCO.CSV
20250324#991310165PMTFLOWCO.CSV
20250321#991310510PMTFLOWCO.CSV
20250319#991310379PMTFLOWCO.CSV
20250330#991310072PMTFLOWCO.CSV
20250321#991310525PMTFLOWCO.CSV
20250330#991310291PMTFLOWCO.CSV
20250401#991310394PMTFLOWCO.CSV
20250322#991310215PMTFLOWCO.CSV
20250320#991310538PMTFLOWCO.CSV
20250325#991310291PMTFLOWCO.CSV
20250326#991310501PMTFLOWCO.CSV
20250320#991310464PMTFLOWCO.CSV
20250321#991310486PMTFLOWCO.CSV
20250402#991310545PMTFLOWCO.CSV
20250317#991310291PMTFLOWCO.CSV
20250325#991310394PMTFLOWCO.CSV
20250323#991410074PMTFLOWCO.CSV
20250325#991310363PMTFLOWCO.CSV
20250328#991310486PMTFLOWCO.CSV
20250320#991310399PMTFLOWCO.CSV
20250330#991310322PMTFLOWCO.CSV
20250331#991410144PMTFLOWCO.CSV
20250402#991310379PMTFLOWCO.CSV
20250330#991310466PMTFLOWCO.CSV
20250324#991310418PMTFLOWCO.CSV
20250321#991310404PMTFLOWCO.CSV
20250402#991310375PMTFLOWCO.CSV
20250402#991310525PMTFLOWCO.CSV
20250322#991310465PMTFLOWCO.CSV
20250322#991310433PMTFLOWCO.CSV
20250320#991304009PMTFLOWCO.CSV
20250324#991310258PMTFLOWCO.CSV
20250325#991410186PMTFLOWCO.CSV
20250329#991310440PMTFLOWCO.CSV
20250319#991303175PMTFLOWCO.CSV
20250321#991310420PMTFLOWCO.CSV
20250401#991310278PMTFLOWCO.CSV
20250321#991310356PMTFLOWCO.CSV
20250402#991310357PMTFLOWCO.CSV
20250329#991310501PMTFLOWCO.CSV
20250324#991310375PMTFLOWCO.CSV
20250331#991310539PMTFLOWCO.CSV
20250323#991310401PMTFLOWCO.CSV
20250330#991410231PMTFLOWCO.CSV
20250323#991310501PMTFLOWCO.CSV
20250327#991310418PMTFLOWCO.CSV
20250321#991310440PMTFLOWCO.CSV
20250319#991310222PMTFLOWCO.CSV
20250331#991310453PMTFLOWCO.CSV
20250320#991310502PMTFLOWCO.CSV
20250327#991310403PMTFLOWCO.CSV
20250322#991310525PMTFLOWCO.CSV
20250401#991410209PMTFLOWCO.CSV
20250323#991410119PMTFLOWCO.CSV
20250331#991404191PMTFLOWCO.CSV
20250329#991310124PMTFLOWCO.CSV
20250328#991310507PMTFLOWCO.CSV
20250331#991310356PMTFLOWCO.CSV
20250329#991310406PMTFLOWCO.CSV
20250319#991310533PMTFLOWCO.CSV
20250327#991404387PMTFLOWCO.CSV
20250321#991310512PMTFLOWCO.CSV
20250329#991410301PMTFLOWCO.CSV
20250326#991310353PMTFLOWCO.CSV
20250325#991310424PMTFLOWCO.CSV
20250328#991310223PMTFLOWCO.CSV
20250327#991310452PMTFLOWCO.CSV
20250327#991310438PMTFLOWCO.CSV
20250328#991310282PMTFLOWCO.CSV
20250327#991310202PMTFLOWCO.CSV
20250327#991310072PMTFLOWCO.CSV
20250322#991310399PMTFLOWCO.CSV
20250401#991310207PMTFLOWCO.CSV
20250331#991310255PMTFLOWCO.CSV
20250326#991310392PMTFLOWCO.CSV
20250401#991310413PMTFLOWCO.CSV
20250326#991310518PMTFLOWCO.CSV
20250328#991310391PMTFLOWCO.CSV
20250324#991310261PMTFLOWCO.CSV
20250329#991410223PMTFLOWCO.CSV
20250322#991410063PMTFLOWCO.CSV
20250326#991310318PMTFLOWCO.CSV
20250322#991310463PMTFLOWCO.CSV
20250325#991310496PMTFLOWCO.CSV
20250323#991310486PMTFLOWCO.CSV
20250325#991310396PMTFLOWCO.CSV
20250329#991310398PMTFLOWCO.CSV
20250402#991310487PMTFLOWCO.CSV
20250325#991310285PMTFLOWCO.CSV
20250321#991310480PMTFLOWCO.CSV
20250331#991310228PMTFLOWCO.CSV
20250320#991310439PMTFLOWCO.CSV
20250319#991310153PMTFLOWCO.CSV
20250324#991310440PMTFLOWCO.CSV
20250320#991410186PMTFLOWCO.CSV
20250322#991310510PMTFLOWCO.CSV
20250323#991310455PMTFLOWCO.CSV
20250331#991310433PMTFLOWCO.CSV
20250330#991404624PMTFLOWCO.CSV
20250330#991310541PMTFLOWCO.CSV
20250329#991310280PMTFLOWCO.CSV
20250328#991310432PMTFLOWCO.CSV
20250323#991310202PMTFLOWCO.CSV
20250321#991310289PMTFLOWCO.CSV
20250326#991310118PMTFLOWCO.CSV
20250319#991310541PMTFLOWCO.CSV'''.split('\n')).to_csv('round.csv',index=False)